# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones
---

<p style="text-align: center;">
<img src=http://www2.famaf.unc.edu.ar/~efernandez/egeo/img/logos/famaf.jpg width=40%>
</p>

 Universidad Nacional de Córdoba

---

## Practico: Aprendizaje No Supervisado
## 1. Preprocesamiento

En la siguiente notebook se presentará las operaciones de preprocesamiento que deben ser aplicadas previo a entrenar los modelos de aprendizaje no supervisado

### Importación de librerías

In [1]:
import pandas as pd 
import numpy as np
import warnings
import pgeocode
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

warnings.filterwarnings('ignore')

### Lectura del dataset 

In [2]:
df = pd.read_parquet('cupones_no_supervisado.parquet')
df

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,EMPDE COMERCIO,SECUNDARIOS,Propia,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,62,92,0
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,SIN DATOS,None,None,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,30,2,0
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,SIN DATOS,PRIMARIOS,Otros,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,29,95,0
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,EMPDE COMERCIO,SECUNDARIOS,Propia,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,41,151,0
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,EMPDE COMERCIO,UNIVERSITARIOS,Alquilada,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,37,87,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124311,0006c15ca823454b68c189da1344d9d7317,0,064194852,NARANJA PLUS RESISTENCIA II,821734,0,PC,1,3,9,...,JUBILADO,SECUNDARIOS,Propia,202105,13210.67,NARANJA PLUS RESISTENCIA II,Sector_No_Operativo,66,105,0
124312,0000ab27a0ed815f947df8bcb834ff97975,0,063299398,PLAN DE CUOTAS SGO DEL ESTERO,152686,0,PC,1,3,3,...,ADMINISTRATIVO,SECUNDARIOS,Propia,202105,121.73,PLAN DE CUOTAS SGO DEL ESTERO,Sector_Empleado_Comercio,74,140,0
124313,0000ab27a0ed815f947df8bcb834ff97975,0,063299398,PLAN DE CUOTAS SGO DEL ESTERO,152687,0,PC,1,3,3,...,ADMINISTRATIVO,SECUNDARIOS,Propia,202105,12284.16,PLAN DE CUOTAS SGO DEL ESTERO,Sector_Empleado_Comercio,74,140,0
124314,000986998f686d538f893956e736fedf611,3,350071053,20 DE MAYO SRL,70,0,PL,1,0,1,...,OPERARIO,SECUNDARIOS,Propia,202002,1040.00,20 DE MAYO SRL,Sector_Operativo,46,261,3


### 1.1 Imputación de datos nulos

In [3]:
df.isna().sum()

dni_titular_movimiento                      0
moneda_movimiento                           0
id_comercio_movimiento                      0
nombre_comercio_histo                       0
numero_cupon_movimiento                     0
debito_credito_movimiento                   0
producto_naranja_movimiento                 0
codigo_empresa_movimiento                   0
tipo_producto_tarjeta_movimiento            0
plan_movimiento                             0
fecha_vto_cupon_movimiento                  0
fecha_presentacion_movimiento               0
fecha_cupon_movimiento                      0
fecha_carga_sistema_movimiento          40715
monto_compra_movimiento                     0
importe_cuota_movimiento                    0
interes_movimiento                          0
cargo_adm_seguro_movimiento                 0
cargo_otorgamiento_movimiento               0
cargo_seguro_vida_movimiento                0
cargo_administrativo_movimiento             0
seleccionado_ng                   

#### Fecha carga sistema movimiento

In [4]:
print('fecha_cupon vs fecha_carga:')
df["fecha_cupon_movimiento"].dt.date.corr(
    df["fecha_carga_sistema_movimiento"].dt.date, method = 'spearman', min_periods = 1)

fecha_cupon vs fecha_carga:


0.9984405676446135

Al calcular su correlación con la columna **fecha_cupon_movimiento**, podemos observar que tienen una relación lineal fuerte, por lo cual decidimos no tener en cuenta esta columna, y si utilizar fecha_cupon_movimiento en su lugar.

#### Tipo_prestamo_movimiento, Nombre_local_histo y Fecha_extraccion_movimiento 

Las variables tipo_prestamo_movimiento, nombre_local_histo y fecha_extraccion_movimiento poseen una gran cantidad de valores nulos (fecha extracción tiene todos sus valores nulos), por lo cuál la mejor decisión es no tenerlas en cuenta.

#### Sexo

In [5]:
(df[df['sexo_descripcion'].isna()])['sexo_descripcion']

220       None
378       None
474       None
485       None
634       None
          ... 
124145    None
124213    None
124214    None
124225    None
124226    None
Name: sexo_descripcion, Length: 647, dtype: object

Procedemos a imputar con 'Sin Datos' a los valores faltantes en la columna sexo_descripcion.

In [6]:
columna = ['sexo_descripcion']
df['sexo_descripcion']= df.sexo_descripcion.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin_Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [7]:
#df['sexo_descripcion'] = df['sexo_descripcion'].astype(str)
df['sexo_descripcion'].value_counts()

Mujer        65541
Hombre       58128
Sin_Datos      647
Name: sexo_descripcion, dtype: int64

#### Datos faltantes Nivel de Estudio

In [8]:
(df[df['nivel_estudio_descripcion_histo'].isna()])['nivel_estudio_descripcion_histo']

1         None
9         None
27        None
30        None
37        None
          ... 
124171    None
124179    None
124181    None
124217    None
124218    None
Name: nivel_estudio_descripcion_histo, Length: 4393, dtype: object

Habiendo analizado en los trabajos anteriores que la pérdida de datos de nivel de estudio también es sistemática, es decir no es posible recuperar este dato, procedemos a imputar dichas filas con Sin datos. 

In [9]:
columna = ['nivel_estudio_descripcion_histo']
df['nivel_estudio_descripcion_histo']= df.nivel_estudio_descripcion_histo.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin_Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [10]:
df.nivel_estudio_descripcion_histo.value_counts()

SECUNDARIOS       66356
PRIMARIOS         30472
TERCIARIOS        14332
UNIVERSITARIOS     8763
Sin_Datos          4393
Name: nivel_estudio_descripcion_histo, dtype: int64

#### Datos faltantes Vivienda

In [11]:
(df[df['rel_vivienda_descripcion_histo'].isna()])['rel_vivienda_descripcion_histo']

1         None
7         None
9         None
15        None
27        None
          ... 
124276    None
124279    None
124280    None
124281    None
124282    None
Name: rel_vivienda_descripcion_histo, Length: 13740, dtype: object

Al igual que el caso anterior, la pérdida de datos de vivienda también es sistemática, es decir no es posible recuperar este dato, procedemos a imputar dichas filas con Sin datos. 

In [12]:
columna = ['rel_vivienda_descripcion_histo']
df["rel_vivienda_descripcion_histo"]= df.rel_vivienda_descripcion_histo.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin_Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [13]:
df.rel_vivienda_descripcion_histo.value_counts()

Propia         85313
Sin_Datos      13740
Otros          10653
De familiar    10473
Alquilada       4137
Name: rel_vivienda_descripcion_histo, dtype: int64

#### Fecha de nacimiento y edad 

In [14]:
(df[df['edad_cliente'].isna()])

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat
220,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202008,0.98,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,237,0
378,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202008,0.98,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,237,0
474,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202009,66.48,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,238,0
634,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202009,66.48,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,238,0
768,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202010,2.32,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,239,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122068,000cde1ad5d8114e7a5dd977930b8835659,3,112000973,LADY STORK STORK MAN,6292,0,PL,1,3,1,...,SIN DATOS,SECUNDARIOS,Propia,202007,5244.57,LADY STORK STORK MAN,Sector_Sin_Datos,<NA>,17,1
124213,000cde1ad5d8114e7a5dd977930b8835659,0,112101030,EL EMPORIO DE LAS GOLOSINAS,1665,0,PL,1,3,1,...,SIN DATOS,SECUNDARIOS,Propia,202007,515.40,EL EMPORIO DE LAS GOLOSINAS,Sector_Sin_Datos,<NA>,17,1
124214,000cde1ad5d8114e7a5dd977930b8835659,0,112101030,EL EMPORIO DE LAS GOLOSINAS,1665,0,PL,1,3,1,...,SIN DATOS,SECUNDARIOS,Propia,202007,515.40,EL EMPORIO DE LAS GOLOSINAS,Sector_Sin_Datos,<NA>,17,1
124225,000650e8144e3a42d47a21aee9e48f34804,3,112142903,ROSARIO VISION OPTICAS,84,0,PL,1,3,1,...,JUBILADO,PRIMARIOS,Propia,202007,2536.23,ROSARIO VISION OPTICAS,Sector_No_Operativo,<NA>,236,1


In [15]:
df_faltante_fech= pd.DataFrame(df[df['edad_cliente'].isna()])
df_faltante_fech.dni_titular_movimiento.value_counts()

000cde1ad5d8114e7a5dd977930b8835659    246
000650e8144e3a42d47a21aee9e48f34804     40
000b0d85fa8a959e2133328830fff990443     20
Name: dni_titular_movimiento, dtype: int64

Habiendo concluido en trabajos anteriores que la falta de la fecha de nacimiento es una pérdida sistemática, no podemos recuperar dicho dato de una forma confiable. En este punto se procederá a imputar el dato faltando de la edad de estos cliente con la edad media, para poder continuar con el preprocesamiento. Sin embargo, sabemos que en los modelos de aprendizaje no supervisado cobra mucha importancia la descripción de los individuos al efecto de poder agrupar los similares, por lo tanto cuando entrenemos dichos modelos tendremos en cuenta que estos tres clientes no tenían el dato de la edad, y los consideraremos como un cluster diferente.  

Otra alternativa es categorizar la variable edad creando rangos, y creando una categoría "Sin datos" para aquellos dnis que no posean edad. 

In [16]:
def to_categorical(column, bin_size=10, min_cut=20, max_cut=70):
    if min_cut is None:
        min_cut = int(round(column.min())) - 1
    value_max = int(np.ceil(column.max()))
    max_cut = min(max_cut, value_max)
    intervals = [(x, x + bin_size) for x in range(min_cut, max_cut, bin_size)]
    if max_cut != value_max:
        intervals.append((max_cut, value_max))
    print(intervals)
    return pd.cut(column, pd.IntervalIndex.from_tuples(intervals))

In [17]:
df['client_edad_cat'] = to_categorical(df.edad_cliente)

[(20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 110)]


In [18]:
columna = ['client_edad_cat']
#df['client_edad_cat']= df.client_edad_cat.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin_Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [19]:
df.client_edad_cat.value_counts()

(30.0, 40.0]     29118
(40.0, 50.0]     28774
(50.0, 60.0]     21052
(60.0, 70.0]     20665
(70.0, 110.0]    14055
(20.0, 30.0]     10269
Sin_Datos          383
Name: client_edad_cat, dtype: int64

Ahora si procedemos a imputar la variable numérica edad. 

In [20]:
columna = ['edad_cliente']
#df['edad_cliente']= df.edad_cliente.fillna(value=np.nan)
imputer = SimpleImputer(missing_values= np.nan, strategy='mean') 
df.loc[ : , columna] = pd.DataFrame(imputer.fit_transform(df.loc[:][columna]) , columns = columna)

#### Datos Faltantes Geograficos de los clientes

In [21]:
col_geo = ['domicilio_codigo_postal', 'pais', 'provincia', 'ciudad', 'domicilio_barrio']
df[col_geo].isna().sum()

domicilio_codigo_postal     957
pais                       3398
provincia                  3424
ciudad                     3885
domicilio_barrio           1145
dtype: int64

##### Reimputación con datos geográficos presentes en el dataset

En primer lugar, procedemos a corregir aquellos errores presentes en el dataset: filas con código postal igual a 0 y con código postal igual a 400.

In [22]:
df[df['domicilio_codigo_postal'].str.len()<4][['pais','provincia','ciudad','domicilio_codigo_postal']].drop_duplicates()

,pais,provincia,ciudad,domicilio_codigo_postal
24863,Argentina,None,None,0
58603,Argentina,TUCUMAN,CAMPO HERRERA,400


In [23]:
print('Cantidad de filas con Cero:', len(df[df['domicilio_codigo_postal']=='0']))

Cantidad de filas con Cero: 26


In [24]:
df['domicilio_codigo_postal'] = df['domicilio_codigo_postal'].replace('0',np.nan)
df[df['domicilio_codigo_postal']=='0']

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,client_edad_cat


In [25]:
df['ciudad'] = df['ciudad'].str.strip()
print('Cantidad de filas con 400 en el CP:',len(df[df['ciudad'].isin(['CAMPO HERRERA'])]))
df[df['ciudad'].isin(['CAMPO HERRERA'])][['pais','provincia','ciudad','domicilio_codigo_postal']].drop_duplicates()

Cantidad de filas con 400 en el CP: 20


,pais,provincia,ciudad,domicilio_codigo_postal
58603,Argentina,TUCUMAN,CAMPO HERRERA,400


Obteniendo el dato de una fuente externa, vamos a reemplazar el código postal de la ciudad Campo Herrera en la provincia Tucumán por el valor correcto.

Fuente: https://codigo-postal.co/argentina/tucuman/campo-herrera/

In [26]:
df.loc[df['ciudad'].isin(['CAMPO HERRERA']),'domicilio_codigo_postal'] = '4105'
print('Cantidad de filas con 400 en el CP:',len(df[df['ciudad'].isin(['CAMPO HERRERA'])]))
df[df['ciudad'].isin(['CAMPO HERRERA'])][['pais','provincia','ciudad','domicilio_codigo_postal']].drop_duplicates()

Cantidad de filas con 400 en el CP: 20


,pais,provincia,ciudad,domicilio_codigo_postal
58603,Argentina,TUCUMAN,CAMPO HERRERA,4105


Recuperaremos las filas donde haya algún dato geográfico presente que permita imputar los faltantes. 

Procedemos a recuperar los datos de **provincia** y **ciudad** a través de **domicilio_codigo_postal** dentro de nuestro dataset.

In [27]:
cp_ciu_nan = df[(df['domicilio_codigo_postal'].isna()!=True) & (df['ciudad'].isna()==True)]['domicilio_codigo_postal'].unique()
df[df['domicilio_codigo_postal'].isin(cp_ciu_nan)][['domicilio_codigo_postal','ciudad']]\
.drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,ciudad
5899,1130,None
3027,1663,SAN MIGUEL
2687,1663,None
2271,2144,None
58610,2506,None
58684,2705,None
17839,2812,None
23486,3230,None
58620,3240,None
116,3300,POSADAS


In [28]:
df.loc[:,'ciudad'] = df['ciudad'].str.upper()
df_dp_cd = df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['ciudad'].isna()==False)]\
            [['domicilio_codigo_postal','ciudad']]\
            .drop_duplicates()\
            .sort_values(by=['domicilio_codigo_postal'])
df_dp_cd = df_dp_cd.rename(columns={'ciudad': 'ciudad_0', 'domicilio_codigo_postal': 'cp'})
df_dp_cd

,cp,ciudad_0
3027,1663,SAN MIGUEL
116,3300,POSADAS
26,3500,RESISTENCIA
870,3503,LA LEONESA
105,4000,SAN MIGUEL DE TUCUMAN
2024,4000,TUCUMAN
174,4101,SAN MIGUEL DE TUCUMAN
795,4101,LAS TALITASTUCUMAN
89,4400,SALTA
100,5000,CORDOBA


Realizamos merge sobre el dataframe creado, e imputamos los datos de ciudad.

In [29]:
df = df.merge(df_dp_cd.drop_duplicates(subset=['cp'], keep='first'), 
              how='left',
              left_on = 'domicilio_codigo_postal', 
              right_on = 'cp')\
            .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,client_edad_cat,ciudad_0
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Propia,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,62.0,92,0,"(60.0, 70.0]",NaN
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Sin_Datos,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,30.0,2,0,"(20.0, 30.0]",NaN
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Otros,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,29.0,95,0,"(20.0, 30.0]",NaN
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Propia,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,41.0,151,0,"(40.0, 50.0]",NaN
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Alquilada,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,37.0,87,0,"(30.0, 40.0]",NaN


In [30]:
# Sustituimos los valores nulos de la columna provincia (provincia_x) por las provincias (en mayus) de la columna
# provincia creada en la anterior unión.
df.loc[df['ciudad'].isnull(), 'ciudad'] = df['ciudad_0'].str.upper()

df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['ciudad'].isna()==True)]\
    [['domicilio_codigo_postal','ciudad','ciudad_0']].drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,ciudad,ciudad_0
5899,1130,NaN,NaN
2271,2144,NaN,NaN
58610,2506,NaN,NaN
58684,2705,NaN,NaN
17839,2812,NaN,NaN
23486,3230,NaN,NaN
58620,3240,NaN,NaN
2111,4146,NaN,NaN
58628,5019,NaN,NaN
494,5101,NaN,NaN


In [31]:
df = df.drop(columns=['ciudad_0'])

Procedemos a imputar **provincia** de la misma manera que **ciudad**.

In [32]:
cp_prov_nan = df[(df['domicilio_codigo_postal'].isna()!=True) & (df['provincia'].isna()==True)]['domicilio_codigo_postal'].unique()
df[df['domicilio_codigo_postal'].isin(cp_prov_nan)][['domicilio_codigo_postal','provincia']]\
.drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,provincia
5899,1130,None
3027,1663,BUENOS AIRES
2687,1663,None
2271,2144,None
58684,2705,None
17839,2812,None
23486,3230,None
116,3300,MISIONES
24727,3300,None
26,3500,CHACO


In [33]:
df.loc[:,'provincia'] = df['provincia'].str.upper()
df_dp_pv = df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['provincia'].isna()==False)]\
            [['domicilio_codigo_postal','provincia']]\
            .drop_duplicates()\
            .sort_values(by=['domicilio_codigo_postal'])
df_dp_pv = df_dp_pv.rename(columns={'provincia': 'provincia_0', 'domicilio_codigo_postal': 'cp'})
df_dp_pv

,cp,provincia_0
3027,1663,BUENOS AIRES
58610,2506,SANTA FE
58620,3240,ENTRE RIOS
116,3300,MISIONES
26,3500,CHACO
870,3503,CHACO
105,4000,TUCUMAN
174,4101,TUCUMAN
77,4400,SALTA
100,5000,CORDOBA


In [34]:
df = df.merge(df_dp_pv.drop_duplicates(subset=['cp'], keep='first'), 
              how='left',
              left_on = 'domicilio_codigo_postal', 
              right_on = 'cp')\
            .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,client_edad_cat,provincia_0
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Propia,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,62.0,92,0,"(60.0, 70.0]",NaN
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Sin_Datos,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,30.0,2,0,"(20.0, 30.0]",NaN
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Otros,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,29.0,95,0,"(20.0, 30.0]",NaN
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Propia,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,41.0,151,0,"(40.0, 50.0]",NaN
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Alquilada,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,37.0,87,0,"(30.0, 40.0]",NaN


In [35]:
# Sustituimos los valores nulos de la columna provincia (provincia_0) por las provincias (en mayus) de la columna
# provincia creada en la anterior unión.

df.loc[df['provincia'].isnull(), 'provincia'] = df['provincia_0'].str.upper()

df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['provincia'].isna()==True)]\
    [['domicilio_codigo_postal','provincia','provincia_0']].drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,provincia,provincia_0
5899,1130,NaN,NaN
2271,2144,NaN,NaN
58684,2705,NaN,NaN
17839,2812,NaN,NaN
23486,3230,NaN,NaN
2111,4146,NaN,NaN
1481,5613,NaN,NaN
2724,8322,NaN,NaN
5466,9015,NaN,NaN


In [36]:
df = df.drop(columns=['provincia_0'])

In [37]:
df[col_geo].isna().sum()

domicilio_codigo_postal     983
pais                       3398
provincia                  1841
ciudad                     2124
domicilio_barrio           1145
dtype: int64

Como podemos observar, existen datos nulos en **provincia** y **ciudad** que persistieron a la imputación anterior, por lo que decidimos buscar en un CSV externo los datos de Códigos Postales, Localidades y Provincias de Argentina.

##### Imputación con dataset externo

In [38]:
df_cp = pd.read_csv('https://raw.githubusercontent.com/JIBarrionuevoGaltier/localidades_AR/master/localidades_cp_maestro.csv')
df_cp.head()

,provincia,id,localidad,cp,id_prov_mstr
0,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,NaN,2
1,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1144.0,2
2,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1145.0,2
3,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1146.0,2
4,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1147.0,2


In [39]:
# Realizamos una curación de datos sobre este dataset.

df_cp.cp = df_cp[df_cp['cp'].isna()!=True]['cp'].apply(int).apply(str)
df_cp.head()

,provincia,id,localidad,cp,id_prov_mstr
0,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,NaN,2
1,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1144,2
2,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1145,2
3,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1146,2
4,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1147,2


Observamos si todos los Códigos Postales de nuestro DataFrame original se encuentran en los objetos del merge.

In [40]:
df_cp = df_cp[~df_cp['cp'].isnull()]
df_cp.isna().sum()

provincia       0
id              0
localidad       0
cp              0
id_prov_mstr    0
dtype: int64

In [41]:
pd.DataFrame({'Provincias': df_cp['provincia'].unique()})\
    .sort_values(by='Provincias',ascending=True)\
    .reset_index().drop(columns=['index'])

,Provincias
0,Buenos Aires
1,Catamarca
2,Chaco
3,Chubut
4,Ciudad Autonoma de Buenos Aires
5,Cordoba
6,Corrientes
7,Entre Rios
8,Formosa
9,Jujuy


In [42]:
df_cp.loc[df_cp['provincia'] == 'Ciudad Autonoma de Buenos Aires', 'provincia'] = 'CAPITAL FEDERAL'
df_cp.loc[df_cp['provincia'] == 'Santiago del Estero', 'provincia'] = 'SGO DEL ESTERO'
df_cp['provincia'].unique()

array(['CAPITAL FEDERAL', 'Buenos Aires', 'Catamarca', 'Cordoba',
       'Corrientes', 'Chaco', 'Chubut', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen',
       'Rio Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'SGO DEL ESTERO', 'Tucuman', 'Tierra del Fuego'],
      dtype=object)

In [43]:
df_cp.loc[:,'provincia'] = df_cp['provincia'].str.upper()
df_cp.loc[:,'localidad'] = df_cp['localidad'].str.upper()
df_cp[['provincia','localidad']].drop_duplicates()

,provincia,localidad
1,CAPITAL FEDERAL,CIUDAD AUTONOMA DE BUENOS AIRES
456,BUENOS AIRES,COLONIA VELEZ
457,BUENOS AIRES,SPURR
458,BUENOS AIRES,SPERONI
459,BUENOS AIRES,SPERATTI
...,...,...
23234,TIERRA DEL FUEGO,ESTANCIA SAN JULIO
23235,TIERRA DEL FUEGO,ESTANCIA SAN JUSTO
23236,TIERRA DEL FUEGO,ESTANCIA SAN MARTIN
23237,TIERRA DEL FUEGO,ESTANCIA RIO EWAN


In [44]:
df_cp = df_cp.rename(columns={'provincia': 'provincia_0'})
df_cp.head()

,provincia_0,id,localidad,cp,id_prov_mstr
1,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1144,2
2,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1145,2
3,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1146,2
4,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1147,2
5,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1148,2


In [45]:
df.shape

(124316, 49)

Procedemos a hacer el merge de los datasets, a través de la columna cp (Codigo Postal)

In [46]:
# Unimos por cp, eliminamos duplicados, conservamos solo cps del dataframe original
df = df.merge(df_cp[['provincia_0','localidad','cp']].drop_duplicates(subset=['cp'], keep='first'), 
                           how='left',
                           left_on = 'domicilio_codigo_postal', 
                           right_on = 'cp')\
                    .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,client_edad_cat,provincia_0,localidad
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,62.0,92,0,"(60.0, 70.0]",BUENOS AIRES,LA TABLADA
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,30.0,2,0,"(20.0, 30.0]",MENDOZA,VILLA NUEVA
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,29.0,95,0,"(20.0, 30.0]",BUENOS AIRES,SANTA TERESITA
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,41.0,151,0,"(40.0, 50.0]",SANTA FE,SAN LORENZO
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,37.0,87,0,"(30.0, 40.0]",CAPITAL FEDERAL,CIUDAD AUTONOMA DE BUENOS AIRES


In [47]:
df.loc[df['provincia'].isnull(), 'provincia'] = df['provincia_0'].str.upper()
df.loc[df['ciudad'].isnull(), 'ciudad'] = df['localidad'].str.upper()

In [48]:
df = df.drop(columns=['provincia_0','localidad'])

In [49]:
df[col_geo].isna().sum()

domicilio_codigo_postal     983
pais                       3398
provincia                   983
ciudad                     1003
domicilio_barrio           1145
dtype: int64

##### Conclusiones y decisiones

* La columna **pais** tiene una sola categoria: 'Argentina'. Sabiendo que todas los registros de la columna **provincia** que tienen datos pertenecen al territorio argentino, se puede descartar ya que no aporta información.
* La columna **domicilio_barrio** no aporta mayor información a la que ya se obtiene con el resto de las columnas geograficas, por lo que tambien se descarta. Ocurre lo mismo con la columna **ciudad**
* La columna **provincia** se utiliza para crear la columna **región** en apartados posteriores. 
* Con los filas con valores faltantes en la columna **domicilio_codigo_postal**, se trabajará de forma similar que con edad, imputando en este caso por el código postal más frecuente, para luego a la hora de entrenar los modelos tener en cuenta estos casos. 

In [ ]:
df_faltante_cod_post= pd.DataFrame(df[df['domicilio_codigo_postal'].isna()])
df_faltante_cod_post.dni_titular_movimiento.value_counts()

In [ ]:
columna = ['domicilio_codigo_postal']
df['domicilio_codigo_postal']= df.domicilio_codigo_postal.fillna(value=np.nan)
imputer = SimpleImputer(missing_values= np.nan, strategy='most_frequent')  
df.loc[ : , columna] = pd.DataFrame(imputer.fit_transform(df.loc[:][columna]) , columns = columna)

#### 1.2 Eliminación de filas duplicadas 

Habiendo imputados todos los valores faltantes, eliminamos aquellas filas que se encuentran duplicadas.

In [52]:
df = df[~df.duplicated()]

In [53]:
df.shape

(65396, 49)

#### 1.3 Valores Atípicos

En este punto, se remite al análisis efectuado en el Trabajo Práctico N° 2 punto 3.2. A diferencia de lo trabajado con los modelos de Aprendizaje Supervisado, en este caso consideramos conveniente quedarnos con aquellas filas con consumos pocos frecuentes, al igual que edad y antigüedad, ya que si bien la probabilidad de aparición en el futuro es baja, son datos que nos permiten conocer mejor a nuestro cliente.  

##### Fechas 

In [54]:
#df.fecha_cupon_movimiento = df.fecha_cupon_movimiento.apply(pd.to_datetime)

In [55]:
print('Cantidad de casos antes de Abril 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-04-01')]))
print('Cantidad de casos antes de Mayo 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-05-01')]))
print('Cantidad de casos antes de Junio 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-06-01')]))
print('Cantidad de casos antes de Julio 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-07-01')]))
print('Cantidad de casos antes de Agosto 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-08-01')]))

Cantidad de casos antes de Abril 2020: 1
Cantidad de casos antes de Mayo 2020: 2
Cantidad de casos antes de Junio 2020: 3
Cantidad de casos antes de Julio 2020: 8
Cantidad de casos antes de Agosto 2020: 910


In [56]:
print('Cantidad de casos despues de Abril 2021:', len(df[df['fecha_cupon_movimiento'] > pd.to_datetime('2021-04-01')]))
print('Cantidad de casos despues de Mayo 2021:', len(df[df['fecha_cupon_movimiento'] > pd.to_datetime('2021-05-01')]))
print('Cantidad de casos despues de Junio 2021:', len(df[df['fecha_cupon_movimiento'] > pd.to_datetime('2021-06-01')]))

Cantidad de casos despues de Abril 2021: 6708
Cantidad de casos despues de Mayo 2021: 453
Cantidad de casos despues de Junio 2021: 0


A pesar de que más adelante se realizará un balanceo de registros en los diferentes meses, observamos que la cantidad de transacciones en los meses previos a Julio del 2020, son demasiados bajos para ser consideramos en nuestros modelos de predicción. 

Haciendo un cálculo similar a los ultimos meses analizados, vemos que Mayo y Abril del 2021 están bien representados, por lo que se decide conservarlos.

In [57]:
df = df[df['fecha_cupon_movimiento'] > pd.to_datetime('2020-07-01')]

In [58]:
df.shape

(65383, 49)

#### 1.4 Variables categoricas (Reagrupación)

##### Estado Civil

In [59]:
df.estado_civil_descripcion.value_counts()

Solteroa               31617
Casadoa                24963
Divorciadoa             2591
Viudoa                  2376
Separacion de hecho     1460
Concubinoa              1438
Sin Datos                863
Novioa                    75
Name: estado_civil_descripcion, dtype: int64

Observamos que existe una alta frecuencia en las categorías Solteros y Casados, perdiendo representativadad en el resto de los estados civiles, por lo cual decidimos agrupar a la mismas en una misma categoría.

In [60]:
soltero = ['Solteroa']
casado = ['Casadoa']
otros = ['Divorciadoa','Viudoa','Concubinoa','Separacion de hecho','Novioa']
sin_datos = ['Sin Datos']

df.loc[df['estado_civil_descripcion'].str.contains('|'.join(soltero)),'estado_civil_cat'] = 'Soltero'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(casado)),'estado_civil_cat'] = 'Casado'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(otros)),'estado_civil_cat'] = 'Otros'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(sin_datos)),'estado_civil_cat'] = 'Sin_datos'

In [61]:
df.estado_civil_cat.value_counts()

Soltero      31617
Casado       24963
Otros         7940
Sin_datos      863
Name: estado_civil_cat, dtype: int64

##### Provincias por Regiones

In [62]:
df.provincia.fillna('Sin_Datos', inplace = True)

In [63]:
df[['provincia']].groupby(by=['provincia']).size()\
                .to_frame()\
                .reset_index(level=['provincia'])\
                .rename(columns = {0:'Cantidad_Transacciones'})\
                .sort_values(by='Cantidad_Transacciones',ascending=False)

,provincia,Cantidad_Transacciones
5,CORDOBA,12798
0,BUENOS AIRES,10337
24,TUCUMAN,4754
20,SANTA FE,4693
12,MENDOZA,4326
4,CHUBUT,2700
3,CHACO,2482
6,CORRIENTES,2451
1,CAPITAL FEDERAL,1974
16,SALTA,1937


Viendo las frecuencias por provincias, vamos a recategorizar sobre una nueva columna, las provincias por regiones.

In [64]:
dic_region = {'REGION_NOROESTE': ['JUJUY','SALTA','TUCUMAN','CATAMARCA','SGO DEL ESTERO'],
              'REGION_NORDESTE': ['CHACO','FORMOSA','CORRIENTES','MISIONES'], 
              'REGION_PAMPEANA': ['CORDOBA','BUENOS AIRES','CAPITAL FEDERAL','ENTRE RIOS','LA PAMPA','SANTA FE'], 
              'REGION_CUYO': ['SAN JUAN','SAN LUIS','LA RIOJA','MENDOZA'], 
              'REGION_PATAGONIA': ['SANTA CRUZ','TIERRA DEL FUEGO','RIO NEGRO','NEUQUEN','CHUBUT'],
              'SIN_DATOS': ['Sin_Datos']}

df['region']= df['provincia']
for i in dic_region:
    df['region'] = df['region'].replace(dic_region[i], i)
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,client_edad_cat,estado_civil_cat,region
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,62.0,92,0,"(60.0, 70.0]",Otros,REGION_PAMPEANA
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,30.0,2,0,"(20.0, 30.0]",Sin_datos,REGION_CUYO
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,29.0,95,0,"(20.0, 30.0]",Soltero,REGION_PAMPEANA
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,41.0,151,0,"(40.0, 50.0]",Soltero,REGION_PAMPEANA
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,37.0,87,0,"(30.0, 40.0]",Soltero,REGION_PAMPEANA


In [65]:
df[['region']].groupby(by=['region']).size()

region
REGION_CUYO          7841
REGION_NORDESTE      7411
REGION_NOROESTE     11132
REGION_PAMPEANA     32047
REGION_PATAGONIA     6453
SIN_DATOS             499
dtype: int64

In [66]:
df.shape

(65383, 51)

#### 1.5 Selección de columnas

Algunas columnas se eliminaron en analisis anteriores. Procedemos a seleccionar las columnas que mejor nos ayudaran a predecir la próxima compra para modelos de aprendizaje no supervisado, donde el objetivo es lograr la mejor caracterización del cliente tipo. 

* **dni_titular_movimiento: Se conserva por ser ID único de cliente**
* **moneda_movimiento: Se conserva para identificar consumos en moneda extranjera**
* id_comercio_movimiento: Se descarta porque conservamos comercio_cat que surge a partir de esta misma variable
* nombre_comercio_histo: Se descarta por usar comercio_cat
* numero_cupon_movimiento: Se descarta por no aportar informacion util a la predicción de la compra
* debito_credito_movimiento: Solo permite diferenciar movimientos positivos de negativos, informacion presente en la variable monto.
* **producto_naranja_movimiento: Se conserva**
* codigo_empresa_movimiento: Se descarta por no aportar informacion util a la predicción de la compra
* **tipo_producto_tarjeta_movimiento: Se conserva**
* plan_movimiento: Se descarta por no aportar informacion util a la predicción de la compra
* fecha_cupon_movimiento: Se elimina porque se conserva año_mes_cupon. 
* monto_compra_movimiento: Se descarta, ya que se conserva monto_ajustado, que es el resultado de su transformación
* importe_cuota_movimiento: Se descarta por su fuerte correlación con monto_compra_movimiento
* interes_movimiento: Se descarta por su falta de datos significativos
* cargo_adm_seguro_movimiento, cargo_otorgamiento_movimiento, cargo_seguro_vida_movimiento, cargo_administrativo_movimiento: Se descartan por tener mayoria de datos en 0.
* seleccionado_ng: Se descarta por no representar nada significativo
* codigo_contable_movimiento: Se descarta por su baja interpretabilidad
* local_venta_producto: Se descarta por aportar la misma información que id_comercio_movimiento
* marca_debito_automatico: Se descarta por no aportar informacion util a la predicción de la compra
* id_comercio_otras_marcas_movimiento: Se descarta por tener mayoria de datos en 0 y una correlación fuerte con id_comercio
* estado_civil_descripcion: Se descarta por recategorización
* **sexo_descripcion: Se conserva**  
* provincia: Se descarta por recategorizacion en regiones
* ciudad: Se descarta para tener un analisis geografico limitado hasta regiones
* **domicilio_codigo_postal:  Se conserva**  
* fecha_de_ingreso_histo: Se descarta porque su único objetivo era calcular la antigüedad del cliente
* fecha_presentacion_movimiento: Se descarta por su fuerte correlación con fecha_cupon_movimiento
* fecha_vto_cupon_movimiento: Se descarta por su fuerte correlación con fecha_cupon_movimiento
* cargo_descripcion_histo: Se descarta por recategorizacion
* **nivel_estudio_descripcion_histo: Se conserva**
* **rel_vivienda_descripcion_histo: Se conserva**
* **client_edad_catSe conserva**

In [67]:
df_final = df[['dni_titular_movimiento', 'moneda_movimiento', 'producto_naranja_movimiento', 'tipo_producto_tarjeta_movimiento',
               'anio_mes_cupon', 'sexo_descripcion', 'monto_ajustado',
               'cargo_sector_desc_hist', 'edad_cliente', 'antig_cliente', 'comercio_cat', 
               'estado_civil_cat', 'region','nivel_estudio_descripcion_histo','rel_vivienda_descripcion_histo',
               'domicilio_codigo_postal', 'fecha_cupon_movimiento','client_edad_cat']]       

In [68]:
df_final.shape

(65383, 18)

#### 1.6 Codificación de variables 

Las variables categóricas relativas a datos personales de los clientes van a ser codificadas utilizando el método mean - encofing, por tal motivo en este apartado procedemos a separarlas del dataset a codificar mediante One - Hot. 

Dichas variables son: 'sexo_descripcion', 'cargo_sector_desc_hist','estado_civil_cat', 'region','nivel_estudio_descripcion_histo','rel_vivienda_descripcion_histo'

In [69]:
df_cliente = df_final[['dni_titular_movimiento', 'sexo_descripcion', 'cargo_sector_desc_hist', 
               'estado_civil_cat', 'region','nivel_estudio_descripcion_histo','rel_vivienda_descripcion_histo', 'client_edad_cat']].drop_duplicates()
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]"
1,000220fa96ec5af89817894033f8099c547,Mujer,Sector_Sin_Datos,Sin_datos,REGION_CUYO,Sin_Datos,Sin_Datos,"(20.0, 30.0]"
2,0002be202de47dfae9cc2304d91161be595,Mujer,Sector_Sin_Datos,Soltero,REGION_PAMPEANA,PRIMARIOS,Otros,"(20.0, 30.0]"
3,000e137d0af42e193be1ff670c00d4d1506,Hombre,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]"
4,0009d010e4faf69552a814a33832b185877,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Alquilada,"(30.0, 40.0]"
...,...,...,...,...,...,...,...,...
58685,000cd61ab767bc36fd3ba59abfb906ce716,Hombre,Sector_Sin_Datos,Casado,REGION_PATAGONIA,SECUNDARIOS,Sin_Datos,"(50.0, 60.0]"
58687,000cc8a8d027dd2166cbc08fe3de32e1388,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Otros,"(40.0, 50.0]"
58688,000793cd3003b11580f438ba2db736fa519,Hombre,Sector_Sin_Datos,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]"
58690,00046eba940afb377b3e92f535de960f790,Hombre,Sector_Sin_Datos,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(20.0, 30.0]"


In [70]:
df_cliente.isna().sum()

dni_titular_movimiento             0
sexo_descripcion                   0
cargo_sector_desc_hist             0
estado_civil_cat                   0
region                             0
nivel_estudio_descripcion_histo    0
rel_vivienda_descripcion_histo     0
client_edad_cat                    0
dtype: int64

In [71]:
df_cliente['client_edad_cat'] = df_cliente['client_edad_cat'].astype(str)

In [72]:
df_cliente.to_parquet('datos_clientes.parquet')

In [73]:
df_final.drop(columns=['sexo_descripcion', 'cargo_sector_desc_hist', 
              'estado_civil_cat', 'region','nivel_estudio_descripcion_histo','rel_vivienda_descripcion_histo','client_edad_cat'], inplace=True)
df_final

,dni_titular_movimiento,moneda_movimiento,producto_naranja_movimiento,tipo_producto_tarjeta_movimiento,anio_mes_cupon,monto_ajustado,edad_cliente,antig_cliente,comercio_cat,domicilio_codigo_postal,fecha_cupon_movimiento
0,0001686b52949b5461ffcbc766687e45031,0,PL,0,202008,5.52,62.0,92,0,1766,2020-08-25
1,000220fa96ec5af89817894033f8099c547,0,PL,0,202008,15.68,30.0,2,0,5521,2020-08-25
2,0002be202de47dfae9cc2304d91161be595,0,PL,0,202008,5.46,29.0,95,0,7107,2020-08-25
3,000e137d0af42e193be1ff670c00d4d1506,0,PL,0,202008,2.50,41.0,151,0,2200,2020-08-25
4,0009d010e4faf69552a814a33832b185877,0,PL,0,202008,2.10,37.0,87,0,1019,2020-08-25
...,...,...,...,...,...,...,...,...,...,...,...
124306,0002716f6d9c2130a8c88a0219de234d459,0,PC,3,202105,9661.01,46.0,93,0,4181,2021-05-14
124309,0001f61dd2845a7e653ebfdaf22dab3b373,0,PC,3,202105,20329.24,48.0,98,0,8300,2021-05-14
124311,0006c15ca823454b68c189da1344d9d7317,0,PC,3,202105,13210.67,66.0,105,0,3503,2021-05-14
124312,0000ab27a0ed815f947df8bcb834ff97975,0,PC,3,202105,121.73,74.0,140,0,4200,2021-05-03


In [74]:
#renombrado de las features
dic_tipo_prod = {42: 'American_express', 44: 'American_express_gold', 0: 'Clasica', 32: 'Master_internacional',
    31: 'Master_nacional', 3: 'Naranja', 4: 'Naranja_virtual', 24: 'Naranja_visa_mini_int.', 23: 'Naranja_visa_mini_nac.',
    12: 'Naranja_visa_internac.', 11: 'Naranja_visa_nacional', -1: 'Sin_Datos', 22: 'Visa_internacional', 21: 'Visa_nacional'}
dic_monedas = {0: 'Pesos',1: 'Dolares',3: 'Zeta'}
df_final.tipo_producto_tarjeta_movimiento = df_final.tipo_producto_tarjeta_movimiento.replace(dic_tipo_prod)
df_final.moneda_movimiento = df_final.moneda_movimiento.replace(dic_monedas)

df_final = df_final.rename({'domicilio_codigo_postal': 'client_domicilio_codigo_postal', 
                            'edad_cliente': 'client_edad', 'antig_cliente': 'client_antig'}, axis=1)

In [75]:
# A efectos de corroborar la cantidad de columnas de nuestra matriz

#print("sexo_descripcion", df_final.sexo_descripcion.nunique())
#print("cargo_sector_desc_hist", df_final.cargo_sector_desc_hist.nunique())
#print("estado_civil_cat", df_final.estado_civil_cat.nunique())
#print("region", df_final.region.nunique())
print("comercio_cat", df_final.comercio_cat.nunique())
print("producto_naranja_movimiento", df_final.producto_naranja_movimiento.nunique())
print("moneda_movimiento", df_final.moneda_movimiento.nunique())
print("tipo_producto_tarjeta_movimiento", df_final.tipo_producto_tarjeta_movimiento.nunique())
#print("nivel_estudio_descripcion_histo", df_final.nivel_estudio_descripcion_histo.nunique())
#print("rel_vivienda_descripcion_histo", df_final.rel_vivienda_descripcion_histo.nunique()) 

comercio_cat 10
producto_naranja_movimiento 12
moneda_movimiento 3
tipo_producto_tarjeta_movimiento 6


In [76]:
ordinal_ft = 'dni_titular_movimiento'
target = 'monto_ajustado'
#target_c = 'fg_aumentado'

# Features numericas
num_features = ['anio_mes_cupon', 'client_edad', 'client_antig', 'client_domicilio_codigo_postal', 'fecha_cupon_movimiento']

# Features categoricas de la transaccion
trans_ft = ['producto_naranja_movimiento', 'tipo_producto_tarjeta_movimiento', 'moneda_movimiento', 'comercio_cat']

# Features categoricas del cliente
#client_ft = ['cat_sexo', 'cat_sect', 'cat_esta_civi','cat_reg','cat_est','cat_viv', 'cat_edad' ]

In [77]:
# Ordenamos por fecha
df_final = df_final.sort_values(by = ['fecha_cupon_movimiento'], ascending = True)
df_cod = df_final.copy()
# Transformacion de fecha a numerica
df_cod['fecha_cupon_movimiento'] = df_cod['fecha_cupon_movimiento'].values.astype(float)/10**11
# Transformacion de tipo de moneda a string
df_cod['moneda_movimiento'] = df_cod['moneda_movimiento'].astype(str)

# Codificación
cat_transformer = OneHotEncoder(handle_unknown='ignore')

encoder = ColumnTransformer(sparse_threshold=0,
    transformers=[
        ('dni', 'drop', [ordinal_ft]),
        ('num', 'passthrough', num_features),
        ('trans', cat_transformer , trans_ft),
        #('client', 'passthrough' , client_ft),
        ('target', 'passthrough', [target])])
df_enc = encoder.fit_transform(df_cod)
cols = encoder.get_feature_names()

In [78]:
# Agregar feature ordinal al principio del dataframe

dff_enc = pd.DataFrame(df_enc, columns=[cols])
cols_o = np.hstack([[ordinal_ft],cols])
df_stack = np.hstack([df_cod[[ordinal_ft]],dff_enc])
df_encode = pd.DataFrame(df_stack , columns=cols_o)

In [79]:
df_encode.shape

(65383, 38)

In [80]:
df_encode.columns

Index(['dni_titular_movimiento', 'anio_mes_cupon', 'client_edad',
       'client_antig', 'client_domicilio_codigo_postal',
       'fecha_cupon_movimiento', 'trans__x0_AV', 'trans__x0_AX',
       'trans__x0_EX', 'trans__x0_MC', 'trans__x0_PC', 'trans__x0_PL',
       'trans__x0_PN', 'trans__x0_PP', 'trans__x0_SM', 'trans__x0_TA',
       'trans__x0_VI', 'trans__x0_ZE', 'trans__x1_American_express',
       'trans__x1_Clasica', 'trans__x1_Master_internacional',
       'trans__x1_Naranja', 'trans__x1_Naranja_virtual',
       'trans__x1_Visa_internacional', 'trans__x2_Dolares', 'trans__x2_Pesos',
       'trans__x2_Zeta', 'trans__x3_0', 'trans__x3_1', 'trans__x3_2',
       'trans__x3_3', 'trans__x3_4', 'trans__x3_5', 'trans__x3_6',
       'trans__x3_7', 'trans__x3_8', 'trans__x3_9', 'monto_ajustado'],
      dtype='object')

In [81]:
df_encode.dni_titular_movimiento.nunique()

844

## 2. Agrupamiento/Balanceo 

### 2.1 Agrupamiento de transacciones por Cliente, Mes, Producto Naranja y Tipo de producto

Procedemos a agrupar las transacciones de los clientes por mes, producto y tipo de producto, debido a que el objeto de esta mentoría es predecir tendencias de consumo mensuales de los diferentes clientes. Aquellas variables que refieren a los datos personales, fueron tomadas por su valor único. 

In [82]:
# Funciones de agregacion para cada columna
aggr = {} 
aggr.update(dict.fromkeys([x for x in cols if 'client' in x], 'max'))
aggr.update(dict.fromkeys([x for x in cols if 'trans' in x], 'sum'))
#aggr.update(dict.fromkeys([x for x in cols if 'cat' in x], 'max'))
aggr.update({target:'sum'})

# Convertimos las columnas categoricas de la transaccion a numericas para poder sumarizarlas
df_encode[num_features + [x for x in cols if 'trans' in x]] = \
df_encode[num_features + [x for x in cols if 'trans' in x]].apply(pd.to_numeric)

# Agrupamiento
group = ['dni_titular_movimiento', 'anio_mes_cupon']

df_mes = df_encode.groupby(group).agg(aggr).reset_index() # edad y antiguedad

In [83]:
df_mes.columns

Index(['dni_titular_movimiento', 'anio_mes_cupon', 'client_edad',
       'client_antig', 'client_domicilio_codigo_postal', 'trans__x0_AV',
       'trans__x0_AX', 'trans__x0_EX', 'trans__x0_MC', 'trans__x0_PC',
       'trans__x0_PL', 'trans__x0_PN', 'trans__x0_PP', 'trans__x0_SM',
       'trans__x0_TA', 'trans__x0_VI', 'trans__x0_ZE',
       'trans__x1_American_express', 'trans__x1_Clasica',
       'trans__x1_Master_internacional', 'trans__x1_Naranja',
       'trans__x1_Naranja_virtual', 'trans__x1_Visa_internacional',
       'trans__x2_Dolares', 'trans__x2_Pesos', 'trans__x2_Zeta', 'trans__x3_0',
       'trans__x3_1', 'trans__x3_2', 'trans__x3_3', 'trans__x3_4',
       'trans__x3_5', 'trans__x3_6', 'trans__x3_7', 'trans__x3_8',
       'trans__x3_9', 'monto_ajustado'],
      dtype='object')

### 2.2 Balanceo de meses 

Corroboramos que existen algunos clientes que no tienen movimientos todos los meses, por lo cuál procedemos a generar esas filas para balancear el dataset

In [84]:
df_mes[['dni_titular_movimiento','anio_mes_cupon']].groupby(['dni_titular_movimiento']).size().value_counts()

10    330
11    188
9     159
5      43
4      24
8      21
1      17
6      17
7      16
3      15
2      14
dtype: int64

In [85]:
print('Cantidad de Meses por Titular (Final):',df_mes['dni_titular_movimiento'].nunique() * df_mes['anio_mes_cupon'].nunique())
print('Shape Dataframe:', df_mes.shape)

Cantidad de Meses por Titular (Final): 10128
Shape Dataframe: (7582, 37)


In [86]:
array_mes = df_mes.anio_mes_cupon.sort_values().unique()
array_dni = df_mes.dni_titular_movimiento.unique()

dic = {'dni': [], 'mes': []}
for dni in array_dni:
    for mes in array_mes:
        dic['dni'].append(dni)
        dic['mes'].append(mes)

df_mes_imp = pd.merge(pd.DataFrame(dic), 
                      df_mes, 
                      left_on=['dni','mes'], 
                      right_on=['dni_titular_movimiento','anio_mes_cupon'], 
                      how='left') \
                .drop(columns=['dni_titular_movimiento','anio_mes_cupon'])
df_mes_imp = df_mes_imp.rename(columns={'dni': 'dni_titular_movimiento','mes': 'anio_mes_cupon'})
df_mes_imp

,dni_titular_movimiento,anio_mes_cupon,client_edad,client_antig,client_domicilio_codigo_postal,trans__x0_AV,trans__x0_AX,trans__x0_EX,trans__x0_MC,trans__x0_PC,...,trans__x3_1,trans__x3_2,trans__x3_3,trans__x3_4,trans__x3_5,trans__x3_6,trans__x3_7,trans__x3_8,trans__x3_9,monto_ajustado
0,000000b5aea2c9ea7cc155f6ebcef97f826,202007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000000b5aea2c9ea7cc155f6ebcef97f826,202008,47.0,225.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.72
2,000000b5aea2c9ea7cc155f6ebcef97f826,202009,47.0,226.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.61
3,000000b5aea2c9ea7cc155f6ebcef97f826,202010,47.0,227.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.46
4,000000b5aea2c9ea7cc155f6ebcef97f826,202011,47.0,228.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10123,000f0b73ebfa002a79a0642b82e87919904,202102,64.0,22.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12622.42
10124,000f0b73ebfa002a79a0642b82e87919904,202103,64.0,23.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2716.90
10125,000f0b73ebfa002a79a0642b82e87919904,202104,64.0,24.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5713.04
10126,000f0b73ebfa002a79a0642b82e87919904,202105,64.0,25.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2063.81


In [87]:
print('Shape Dataframe:', df_mes.shape)

Shape Dataframe: (7582, 37)


In [88]:
str_dni_tit = 'dni_titular_movimiento'
str_mes = 'anio_mes_cupon'

# Imputamos 0 en las columnas de la transacción que tienen nan
cols = df_mes_imp.columns.values
cols_zero = [x for x in cols if 'trans' in x]
cols_zero.append('monto_ajustado')
df_mes_imp[cols_zero] = df_mes_imp[cols_zero].fillna(0)

# Imputamos lo valores propios del cliente
cols_client = [x for x in cols if ('client_' in x) & ('antig' not in x) & ('edad' not in x)]
cols_client_imp = cols_client.copy()
cols_client_imp.append(str_dni_tit)

data_mes = df_mes_imp[cols_client_imp].dropna().drop_duplicates().reset_index().drop(columns='index')
df_imputer = df_mes_imp.drop(columns=cols_client).copy()

# Imputamos los datos del cliente y ordenamos
df_mes_imp = df_imputer.merge(data_mes, on=str_dni_tit)[cols].sort_values(by=[ str_dni_tit , str_mes ]).reset_index().drop(columns=['index'])

# Imputamos antigüedad
for dni in array_dni:
    min_mes = df_mes_imp[(df_mes_imp['dni_titular_movimiento'] == dni) & (~df_mes_imp.client_antig.isna())]['anio_mes_cupon'].min()
    min_ant = df_mes_imp[(df_mes_imp['dni_titular_movimiento'] == dni) & (~df_mes_imp.client_antig.isna())]['client_antig'].min()
    for mes in array_mes:
        num_res = array_mes.tolist().index(mes) - array_mes.tolist().index(min_mes)
        antig_cli = min_ant + num_res
        if antig_cli < 0:
            df_mes_imp.loc[((df_mes_imp['dni_titular_movimiento'] == dni) & (df_mes_imp['anio_mes_cupon'] == mes)), 'client_antig'] = 0
        else:
            df_mes_imp.loc[((df_mes_imp['dni_titular_movimiento'] == dni) & (df_mes_imp['anio_mes_cupon'] == mes)), 'client_antig'] = antig_cli

In [89]:
# Imputación Edad
df_mes_imp_eb = df_mes_imp[['dni_titular_movimiento', 'anio_mes_cupon', 'client_edad']].copy()
for dni in array_dni:
    edad = df_mes_imp_eb[df_mes_imp['dni_titular_movimiento'] == dni]['client_edad'].fillna(method='backfill')
    df_mes_imp_eb.loc[((df_mes_imp['dni_titular_movimiento'] == dni) & (df_mes_imp['client_edad'].isna())), 'client_edad'] = edad
df_mes_imp_eb = df_mes_imp_eb.rename(columns={'client_edad': 'edad_b'})

df_mes_imp_ef = df_mes_imp[['dni_titular_movimiento', 'anio_mes_cupon', 'client_edad']].copy()
for dni in array_dni:
    edad = df_mes_imp_ef[df_mes_imp['dni_titular_movimiento'] == dni]['client_edad'].fillna(method='ffill')
    df_mes_imp_ef.loc[((df_mes_imp['dni_titular_movimiento'] == dni) & (df_mes_imp['client_edad'].isna())), 'client_edad'] = edad
df_mes_imp_ef = df_mes_imp_ef.rename(columns={'client_edad': 'edad_f'})

df_mes_imp = df_mes_imp.merge(df_mes_imp_eb,
                             left_on=['dni_titular_movimiento', 'anio_mes_cupon'],
                             right_on=['dni_titular_movimiento', 'anio_mes_cupon']
                            )
df_mes_imp = df_mes_imp.merge(df_mes_imp_ef,
                             left_on=['dni_titular_movimiento', 'anio_mes_cupon'],
                             right_on=['dni_titular_movimiento', 'anio_mes_cupon']
                            )
df_mes_imp.loc[df_mes_imp['client_edad'].isna(), 'client_edad'] = df_mes_imp['edad_b']
df_mes_imp.loc[df_mes_imp['client_edad'].isna(), 'client_edad'] = df_mes_imp['edad_f']

df_mes_imp.drop(columns=['edad_b','edad_f'], inplace=True)

In [90]:
df_mes = df_mes_imp.copy()
df_mes

,dni_titular_movimiento,anio_mes_cupon,client_edad,client_antig,client_domicilio_codigo_postal,trans__x0_AV,trans__x0_AX,trans__x0_EX,trans__x0_MC,trans__x0_PC,...,trans__x3_1,trans__x3_2,trans__x3_3,trans__x3_4,trans__x3_5,trans__x3_6,trans__x3_7,trans__x3_8,trans__x3_9,monto_ajustado
0,000000b5aea2c9ea7cc155f6ebcef97f826,202007,47.0,224.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,000000b5aea2c9ea7cc155f6ebcef97f826,202008,47.0,225.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.72
2,000000b5aea2c9ea7cc155f6ebcef97f826,202009,47.0,226.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.61
3,000000b5aea2c9ea7cc155f6ebcef97f826,202010,47.0,227.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.46
4,000000b5aea2c9ea7cc155f6ebcef97f826,202011,47.0,228.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10123,000f0b73ebfa002a79a0642b82e87919904,202102,64.0,22.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12622.42
10124,000f0b73ebfa002a79a0642b82e87919904,202103,64.0,23.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2716.90
10125,000f0b73ebfa002a79a0642b82e87919904,202104,64.0,24.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5713.04
10126,000f0b73ebfa002a79a0642b82e87919904,202105,64.0,25.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2063.81


In [91]:
df_mes.columns

Index(['dni_titular_movimiento', 'anio_mes_cupon', 'client_edad',
       'client_antig', 'client_domicilio_codigo_postal', 'trans__x0_AV',
       'trans__x0_AX', 'trans__x0_EX', 'trans__x0_MC', 'trans__x0_PC',
       'trans__x0_PL', 'trans__x0_PN', 'trans__x0_PP', 'trans__x0_SM',
       'trans__x0_TA', 'trans__x0_VI', 'trans__x0_ZE',
       'trans__x1_American_express', 'trans__x1_Clasica',
       'trans__x1_Master_internacional', 'trans__x1_Naranja',
       'trans__x1_Naranja_virtual', 'trans__x1_Visa_internacional',
       'trans__x2_Dolares', 'trans__x2_Pesos', 'trans__x2_Zeta', 'trans__x3_0',
       'trans__x3_1', 'trans__x3_2', 'trans__x3_3', 'trans__x3_4',
       'trans__x3_5', 'trans__x3_6', 'trans__x3_7', 'trans__x3_8',
       'trans__x3_9', 'monto_ajustado'],
      dtype='object')

## 3. Creación de Nuevos Atributos

### 3.1 Creación de variable categórica

Creamos una columna que indica si un cliente ha aumentado su consumo personal mes a mes. El mes inicial para cada cliente se inicializa en 0.

Cabe destacar que a la hora de agrupar los movimientos en cada mes, se considera la fecha de cierre de la tarjeta, por lo cual cada mes comprende desde el dia 25 del mes anterior hasta el dia 24 del corriente, inclusive.

In [92]:
df_dni_mes_mon = df_mes[['dni_titular_movimiento', 'anio_mes_cupon','monto_ajustado']] \
                        .groupby(['dni_titular_movimiento', 'anio_mes_cupon']).sum('monto_ajustado') \
                        .reset_index() \
                        .sort_values(by=['dni_titular_movimiento', 'anio_mes_cupon'])
df_dni_mes_mon

,dni_titular_movimiento,anio_mes_cupon,monto_ajustado
0,000000b5aea2c9ea7cc155f6ebcef97f826,202007,0.00
1,000000b5aea2c9ea7cc155f6ebcef97f826,202008,-4.72
2,000000b5aea2c9ea7cc155f6ebcef97f826,202009,-4.61
3,000000b5aea2c9ea7cc155f6ebcef97f826,202010,-4.46
4,000000b5aea2c9ea7cc155f6ebcef97f826,202011,-4.34
...,...,...,...
10123,000f0b73ebfa002a79a0642b82e87919904,202102,12622.42
10124,000f0b73ebfa002a79a0642b82e87919904,202103,2716.90
10125,000f0b73ebfa002a79a0642b82e87919904,202104,5713.04
10126,000f0b73ebfa002a79a0642b82e87919904,202105,2063.81


In [93]:
# Agregación de la nueva columna seteada en 0.
# En la iteración, se le cambia el valor a 1 en caso de que cumpla con las condiciones preestablecidas.

df_dni_mes_mon['fg_aumentado'] = 0
for i in range(1,len(df_dni_mes_mon)):
    if (df_dni_mes_mon.iloc[i]['dni_titular_movimiento'] == df_dni_mes_mon.iloc[i-1]['dni_titular_movimiento']) \
        & (df_dni_mes_mon.iloc[i]['anio_mes_cupon'] > df_dni_mes_mon.iloc[i-1]['anio_mes_cupon']):
            var_mes = df_dni_mes_mon.iloc[i]['monto_ajustado'] - df_dni_mes_mon.iloc[i-1]['monto_ajustado']
            if (var_mes > (abs(df_dni_mes_mon.iloc[i-1]['monto_ajustado'])*0.1)):
                    df_dni_mes_mon.iloc[i,3] = 1

In [94]:
df_dni_mes_mon.head(20)

,dni_titular_movimiento,anio_mes_cupon,monto_ajustado,fg_aumentado
0,000000b5aea2c9ea7cc155f6ebcef97f826,202007,0.00,0
1,000000b5aea2c9ea7cc155f6ebcef97f826,202008,-4.72,0
2,000000b5aea2c9ea7cc155f6ebcef97f826,202009,-4.61,0
3,000000b5aea2c9ea7cc155f6ebcef97f826,202010,-4.46,0
4,000000b5aea2c9ea7cc155f6ebcef97f826,202011,-4.34,0
5,000000b5aea2c9ea7cc155f6ebcef97f826,202012,-4.21,0
6,000000b5aea2c9ea7cc155f6ebcef97f826,202101,-4.08,0
7,000000b5aea2c9ea7cc155f6ebcef97f826,202102,-3.97,0
8,000000b5aea2c9ea7cc155f6ebcef97f826,202103,-3.83,0
9,000000b5aea2c9ea7cc155f6ebcef97f826,202104,-3.72,0


In [95]:
df_dni_mes_mon = df_dni_mes_mon.drop(columns=['monto_ajustado'])
df_dni_mes_mon = df_dni_mes_mon.rename(columns={'dni_titular_movimiento': 'dni_titular_movimiento_c', 'anio_mes_cupon': 'anio_mes_cupon_c'})
df_dni_mes_mon

,dni_titular_movimiento_c,anio_mes_cupon_c,fg_aumentado
0,000000b5aea2c9ea7cc155f6ebcef97f826,202007,0
1,000000b5aea2c9ea7cc155f6ebcef97f826,202008,0
2,000000b5aea2c9ea7cc155f6ebcef97f826,202009,0
3,000000b5aea2c9ea7cc155f6ebcef97f826,202010,0
4,000000b5aea2c9ea7cc155f6ebcef97f826,202011,0
...,...,...,...
10123,000f0b73ebfa002a79a0642b82e87919904,202102,0
10124,000f0b73ebfa002a79a0642b82e87919904,202103,0
10125,000f0b73ebfa002a79a0642b82e87919904,202104,1
10126,000f0b73ebfa002a79a0642b82e87919904,202105,0


In [96]:
df_obj = df_mes.merge( df_dni_mes_mon, 
             left_on=['dni_titular_movimiento', 'anio_mes_cupon'], 
             right_on=['dni_titular_movimiento_c', 'anio_mes_cupon_c']) \
        .drop(columns= ['dni_titular_movimiento_c','anio_mes_cupon_c'])

In [97]:
df_obj_diario = df_encode.merge( df_dni_mes_mon, 
             left_on=['dni_titular_movimiento', 'anio_mes_cupon'], 
             right_on=['dni_titular_movimiento_c', 'anio_mes_cupon_c']) \
        .drop(columns= ['dni_titular_movimiento_c','anio_mes_cupon_c'])

In [98]:
df_obj.shape

(10128, 38)

In [99]:
df_obj['dni_titular_movimiento'].nunique()

844

### 3.2 Creación de variable mes ordinal

Creamos una variable mes que nos permite mantener el orden en las transacciones realizadas por los contribuyentes. 

In [100]:
df_obj['o_mes'] = df_obj.anio_mes_cupon.astype('category').cat.codes
df_obj    

,dni_titular_movimiento,anio_mes_cupon,client_edad,client_antig,client_domicilio_codigo_postal,trans__x0_AV,trans__x0_AX,trans__x0_EX,trans__x0_MC,trans__x0_PC,...,trans__x3_3,trans__x3_4,trans__x3_5,trans__x3_6,trans__x3_7,trans__x3_8,trans__x3_9,monto_ajustado,fg_aumentado,o_mes
0,000000b5aea2c9ea7cc155f6ebcef97f826,202007,47.0,224.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0,0
1,000000b5aea2c9ea7cc155f6ebcef97f826,202008,47.0,225.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.72,0,1
2,000000b5aea2c9ea7cc155f6ebcef97f826,202009,47.0,226.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.61,0,2
3,000000b5aea2c9ea7cc155f6ebcef97f826,202010,47.0,227.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.46,0,3
4,000000b5aea2c9ea7cc155f6ebcef97f826,202011,47.0,228.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.34,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10123,000f0b73ebfa002a79a0642b82e87919904,202102,64.0,22.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12622.42,0,7
10124,000f0b73ebfa002a79a0642b82e87919904,202103,64.0,23.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2716.90,0,8
10125,000f0b73ebfa002a79a0642b82e87919904,202104,64.0,24.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5713.04,1,9
10126,000f0b73ebfa002a79a0642b82e87919904,202105,64.0,25.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2063.81,0,10


### 3.4 Mean Encofing 

Recordamos que aun no hemos categorizado las caracteristicas personales de los clientes. Ahora vamos a realizar una codificación de esos datos utilizando la variable target fg_aumentado

#### Cargo_sector_desc_hist

In [101]:
df_cliente.cargo_sector_desc_hist.value_counts()

Sector_Sin_Datos            475
Sector_Empleado_Comercio    150
Sector_No_Operativo          71
Sector_Operativo             70
Sector_Educativo             44
Sector_Seguridad             23
Sector_Financiero             8
Sector_Salud                  3
Name: cargo_sector_desc_hist, dtype: int64

In [102]:
#calculamos la suma de target para cada dni
step1 = df_obj[['dni_titular_movimiento', 'fg_aumentado']].groupby(['dni_titular_movimiento']).sum().reset_index().rename({'fg_aumentado' : 'sum_of_target'}, axis=1)
step1

,dni_titular_movimiento,sum_of_target
0,000000b5aea2c9ea7cc155f6ebcef97f826,1
1,00000b8415f19846d12e852ebd0d52ec432,4
2,00000d109ca5a0c6b3e76295f0292c38189,1
3,000017309738c042c2ec144d5d3d3632673,6
4,00001b02afa8956b08af05c339039882046,3
...,...,...
839,000ef071e95351afc1149fa1afe4862a079,5
840,000ef0bc6859b57ec9cc7db583dd73ca697,6
841,000ef64a5f2323a3979805450f2e890c495,3
842,000eff02871f717d5c860eae9e0347a5523,5


In [103]:
df_cliente = df_cliente.merge(step1, 
             left_on=['dni_titular_movimiento'], 
             right_on=['dni_titular_movimiento',])
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7
1,000220fa96ec5af89817894033f8099c547,Mujer,Sector_Sin_Datos,Sin_datos,REGION_CUYO,Sin_Datos,Sin_Datos,"(20.0, 30.0]",4
2,0002be202de47dfae9cc2304d91161be595,Mujer,Sector_Sin_Datos,Soltero,REGION_PAMPEANA,PRIMARIOS,Otros,"(20.0, 30.0]",5
3,000e137d0af42e193be1ff670c00d4d1506,Hombre,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]",4
4,0009d010e4faf69552a814a33832b185877,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Alquilada,"(30.0, 40.0]",4
...,...,...,...,...,...,...,...,...,...
839,000cd61ab767bc36fd3ba59abfb906ce716,Hombre,Sector_Sin_Datos,Casado,REGION_PATAGONIA,SECUNDARIOS,Sin_Datos,"(50.0, 60.0]",1
840,000cc8a8d027dd2166cbc08fe3de32e1388,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Otros,"(40.0, 50.0]",1
841,000793cd3003b11580f438ba2db736fa519,Hombre,Sector_Sin_Datos,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]",1
842,00046eba940afb377b3e92f535de960f790,Hombre,Sector_Sin_Datos,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(20.0, 30.0]",1


In [104]:
df_cliente.isna().sum()

dni_titular_movimiento             0
sexo_descripcion                   0
cargo_sector_desc_hist             0
estado_civil_cat                   0
region                             0
nivel_estudio_descripcion_histo    0
rel_vivienda_descripcion_histo     0
client_edad_cat                    0
sum_of_target                      0
dtype: int64

In [105]:
step2 = df_cliente[['cargo_sector_desc_hist', 'dni_titular_movimiento']].groupby(['cargo_sector_desc_hist']).size().reset_index(name='count_of_target').sort_values(by='count_of_target')
step2

,cargo_sector_desc_hist,count_of_target
5,Sector_Salud,3
2,Sector_Financiero,8
6,Sector_Seguridad,23
0,Sector_Educativo,44
4,Sector_Operativo,70
3,Sector_No_Operativo,71
1,Sector_Empleado_Comercio,150
7,Sector_Sin_Datos,475


In [106]:
step1vs = df_cliente[['cargo_sector_desc_hist', 'sum_of_target']].groupby(['cargo_sector_desc_hist']).sum().reset_index()
step1vs

,cargo_sector_desc_hist,sum_of_target
0,Sector_Educativo,181
1,Sector_Empleado_Comercio,587
2,Sector_Financiero,29
3,Sector_No_Operativo,284
4,Sector_Operativo,264
5,Sector_Salud,7
6,Sector_Seguridad,84
7,Sector_Sin_Datos,1701


In [107]:
step1vs = df_cliente[['cargo_sector_desc_hist', 'sum_of_target']].groupby(['cargo_sector_desc_hist']).sum().reset_index()
step1vs

,cargo_sector_desc_hist,sum_of_target
0,Sector_Educativo,181
1,Sector_Empleado_Comercio,587
2,Sector_Financiero,29
3,Sector_No_Operativo,284
4,Sector_Operativo,264
5,Sector_Salud,7
6,Sector_Seguridad,84
7,Sector_Sin_Datos,1701


In [108]:
step3 =  step2.merge(step1vs, 
             left_on=['cargo_sector_desc_hist'], 
             right_on=['cargo_sector_desc_hist',])
step3 = pd.DataFrame(step3)
step3

,cargo_sector_desc_hist,count_of_target,sum_of_target
0,Sector_Salud,3,7
1,Sector_Financiero,8,29
2,Sector_Seguridad,23,84
3,Sector_Educativo,44,181
4,Sector_Operativo,70,264
5,Sector_No_Operativo,71,284
6,Sector_Empleado_Comercio,150,587
7,Sector_Sin_Datos,475,1701


In [109]:
df_cliente['sum_of_target'].sum()

3137

In [110]:
df_cliente.dni_titular_movimiento.nunique()

844

In [111]:
print(step3['count_of_target'].sum(), step3['sum_of_target'].sum())

844 3137


In [112]:
step3['mean_cargo'] = (step3.sum_of_target / step3.count_of_target).round(2)
step3

,cargo_sector_desc_hist,count_of_target,sum_of_target,mean_cargo
0,Sector_Salud,3,7,2.33
1,Sector_Financiero,8,29,3.62
2,Sector_Seguridad,23,84,3.65
3,Sector_Educativo,44,181,4.11
4,Sector_Operativo,70,264,3.77
5,Sector_No_Operativo,71,284,4.00
6,Sector_Empleado_Comercio,150,587,3.91
7,Sector_Sin_Datos,475,1701,3.58


In [113]:
df_cliente = df_cliente.merge(step3, 
             left_on=['cargo_sector_desc_hist'], 
             right_on=['cargo_sector_desc_hist',])
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target_x,count_of_target,sum_of_target_y,mean_cargo
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,150,587,3.91
1,000e137d0af42e193be1ff670c00d4d1506,Hombre,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]",4,150,587,3.91
2,0009d010e4faf69552a814a33832b185877,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Alquilada,"(30.0, 40.0]",4,150,587,3.91
3,000997fe351702567b1e14d472bf53db592,Mujer,Sector_Empleado_Comercio,Casado,REGION_CUYO,SECUNDARIOS,Otros,"(30.0, 40.0]",4,150,587,3.91
4,000167afba54ed80262018628108b035962,Mujer,Sector_Empleado_Comercio,Casado,REGION_PAMPEANA,SECUNDARIOS,Propia,"(30.0, 40.0]",4,150,587,3.91
...,...,...,...,...,...,...,...,...,...,...,...,...
839,0000872cfa422565e32ef1f5fc95f841894,Hombre,Sector_Financiero,Casado,REGION_PAMPEANA,PRIMARIOS,Propia,"(60.0, 70.0]",3,8,29,3.62
840,000d75b3318c54c2829fbc59fc3b568e418,Hombre,Sector_Financiero,Casado,REGION_NORDESTE,SECUNDARIOS,Propia,"(50.0, 60.0]",3,8,29,3.62
841,000b713b910e2355799d02ea2140bc36680,Hombre,Sector_Financiero,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(70.0, 110.0]",4,8,29,3.62
842,000843ab221e7409b06b518c1d91cfe2857,Hombre,Sector_Financiero,Casado,REGION_PATAGONIA,SECUNDARIOS,Propia,"(70.0, 110.0]",4,8,29,3.62


In [114]:
df_cliente.isna().sum()

dni_titular_movimiento             0
sexo_descripcion                   0
cargo_sector_desc_hist             0
estado_civil_cat                   0
region                             0
nivel_estudio_descripcion_histo    0
rel_vivienda_descripcion_histo     0
client_edad_cat                    0
sum_of_target_x                    0
count_of_target                    0
sum_of_target_y                    0
mean_cargo                         0
dtype: int64

In [115]:
df_cliente = df_cliente.drop(columns= ['count_of_target','sum_of_target_y']).rename({'sum_of_target_x': 'sum_of_target'}, axis=1)
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target,mean_cargo
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91
1,000e137d0af42e193be1ff670c00d4d1506,Hombre,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]",4,3.91
2,0009d010e4faf69552a814a33832b185877,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Alquilada,"(30.0, 40.0]",4,3.91
3,000997fe351702567b1e14d472bf53db592,Mujer,Sector_Empleado_Comercio,Casado,REGION_CUYO,SECUNDARIOS,Otros,"(30.0, 40.0]",4,3.91
4,000167afba54ed80262018628108b035962,Mujer,Sector_Empleado_Comercio,Casado,REGION_PAMPEANA,SECUNDARIOS,Propia,"(30.0, 40.0]",4,3.91
...,...,...,...,...,...,...,...,...,...,...
839,0000872cfa422565e32ef1f5fc95f841894,Hombre,Sector_Financiero,Casado,REGION_PAMPEANA,PRIMARIOS,Propia,"(60.0, 70.0]",3,3.62
840,000d75b3318c54c2829fbc59fc3b568e418,Hombre,Sector_Financiero,Casado,REGION_NORDESTE,SECUNDARIOS,Propia,"(50.0, 60.0]",3,3.62
841,000b713b910e2355799d02ea2140bc36680,Hombre,Sector_Financiero,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(70.0, 110.0]",4,3.62
842,000843ab221e7409b06b518c1d91cfe2857,Hombre,Sector_Financiero,Casado,REGION_PATAGONIA,SECUNDARIOS,Propia,"(70.0, 110.0]",4,3.62


#### Sexo_descripcion

In [116]:
df_cliente.sexo_descripcion.value_counts()

Mujer        449
Hombre       388
Sin_Datos      7
Name: sexo_descripcion, dtype: int64

Remitimos al step1 creado para la variable anterior, que contiene la suma de la variable target por cada dni_titular_movimiento.

In [117]:
step2 = df_cliente[['sexo_descripcion', 'dni_titular_movimiento']].groupby(['sexo_descripcion']).size().reset_index(name='count_of_target').sort_values(by='count_of_target')
step2

,sexo_descripcion,count_of_target
2,Sin_Datos,7
0,Hombre,388
1,Mujer,449


In [118]:
step1vs = df_cliente[['sexo_descripcion', 'sum_of_target']].groupby(['sexo_descripcion']).sum().reset_index()
step1vs

,sexo_descripcion,sum_of_target
0,Hombre,1393
1,Mujer,1726
2,Sin_Datos,18


In [119]:
step3 =  step2.merge(step1vs, 
             left_on=['sexo_descripcion'], 
             right_on=['sexo_descripcion',])
step3 = pd.DataFrame(step3)
step3

,sexo_descripcion,count_of_target,sum_of_target
0,Sin_Datos,7,18
1,Hombre,388,1393
2,Mujer,449,1726


In [120]:
df_cliente['sum_of_target'].sum()

3137

In [121]:
df_cliente.dni_titular_movimiento.nunique()

844

In [122]:
print(step3['count_of_target'].sum(), step3['sum_of_target'].sum())

844 3137


In [123]:
step3['mean_sexo'] = (step3.sum_of_target / step3.count_of_target).round(2)
step3

,sexo_descripcion,count_of_target,sum_of_target,mean_sexo
0,Sin_Datos,7,18,2.57
1,Hombre,388,1393,3.59
2,Mujer,449,1726,3.84


In [124]:
df_cliente = df_cliente.merge(step3, 
             left_on=['sexo_descripcion'], 
             right_on=['sexo_descripcion',])
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target_x,mean_cargo,count_of_target,sum_of_target_y,mean_sexo
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,388,1393,3.59
1,000e137d0af42e193be1ff670c00d4d1506,Hombre,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]",4,3.91,388,1393,3.59
2,0008995b3880f8e8e98090dc1c753e94553,Hombre,Sector_Empleado_Comercio,Casado,REGION_NOROESTE,SECUNDARIOS,Propia,"(50.0, 60.0]",5,3.91,388,1393,3.59
3,000190c81ee107511f72f56e58ca3923057,Hombre,Sector_Empleado_Comercio,Casado,REGION_PAMPEANA,TERCIARIOS,Propia,"(40.0, 50.0]",4,3.91,388,1393,3.59
4,000c49ccbb959a141c075be6018d8652091,Hombre,Sector_Empleado_Comercio,Soltero,REGION_NOROESTE,SECUNDARIOS,Propia,"(40.0, 50.0]",4,3.91,388,1393,3.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,0006e947a6f89a8bcb835ae85c60ff93526,Sin_Datos,Sector_Sin_Datos,Soltero,REGION_NORDESTE,Sin_Datos,Sin_Datos,"(30.0, 40.0]",2,3.58,7,18,2.57
840,00017733df45bb08ee7abd44e6037cf7471,Sin_Datos,Sector_Sin_Datos,Soltero,REGION_NOROESTE,Sin_Datos,Sin_Datos,"(30.0, 40.0]",3,3.58,7,18,2.57
841,000e5f85c3a5e3fe6dc1f214f01f55ae617,Sin_Datos,Sector_Sin_Datos,Soltero,REGION_NORDESTE,Sin_Datos,Sin_Datos,"(20.0, 30.0]",2,3.58,7,18,2.57
842,000650e8144e3a42d47a21aee9e48f34804,Sin_Datos,Sector_No_Operativo,Sin_datos,SIN_DATOS,PRIMARIOS,Propia,Sin_Datos,2,4.00,7,18,2.57


In [125]:
df_cliente = df_cliente.drop(columns= ['count_of_target','sum_of_target_y']).rename({'sum_of_target_x': 'sum_of_target'}, axis=1)
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target,mean_cargo,mean_sexo
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59
1,000e137d0af42e193be1ff670c00d4d1506,Hombre,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]",4,3.91,3.59
2,0008995b3880f8e8e98090dc1c753e94553,Hombre,Sector_Empleado_Comercio,Casado,REGION_NOROESTE,SECUNDARIOS,Propia,"(50.0, 60.0]",5,3.91,3.59
3,000190c81ee107511f72f56e58ca3923057,Hombre,Sector_Empleado_Comercio,Casado,REGION_PAMPEANA,TERCIARIOS,Propia,"(40.0, 50.0]",4,3.91,3.59
4,000c49ccbb959a141c075be6018d8652091,Hombre,Sector_Empleado_Comercio,Soltero,REGION_NOROESTE,SECUNDARIOS,Propia,"(40.0, 50.0]",4,3.91,3.59
...,...,...,...,...,...,...,...,...,...,...,...
839,0006e947a6f89a8bcb835ae85c60ff93526,Sin_Datos,Sector_Sin_Datos,Soltero,REGION_NORDESTE,Sin_Datos,Sin_Datos,"(30.0, 40.0]",2,3.58,2.57
840,00017733df45bb08ee7abd44e6037cf7471,Sin_Datos,Sector_Sin_Datos,Soltero,REGION_NOROESTE,Sin_Datos,Sin_Datos,"(30.0, 40.0]",3,3.58,2.57
841,000e5f85c3a5e3fe6dc1f214f01f55ae617,Sin_Datos,Sector_Sin_Datos,Soltero,REGION_NORDESTE,Sin_Datos,Sin_Datos,"(20.0, 30.0]",2,3.58,2.57
842,000650e8144e3a42d47a21aee9e48f34804,Sin_Datos,Sector_No_Operativo,Sin_datos,SIN_DATOS,PRIMARIOS,Propia,Sin_Datos,2,4.00,2.57


#### Estado_civil_cat

In [126]:
df_cliente.estado_civil_cat.value_counts()

Soltero      434
Casado       295
Otros         98
Sin_datos     17
Name: estado_civil_cat, dtype: int64

Remitimos al step1 creado para la primer variable, que contiene la suma de la variable target por cada dni_titular_movimiento.

In [127]:
step2 = df_cliente[['estado_civil_cat', 'dni_titular_movimiento']].groupby(['estado_civil_cat']).size().reset_index(name='count_of_target').sort_values(by='count_of_target')
step2

,estado_civil_cat,count_of_target
2,Sin_datos,17
1,Otros,98
0,Casado,295
3,Soltero,434


In [128]:
step1vs = df_cliente[['estado_civil_cat', 'sum_of_target']].groupby(['estado_civil_cat']).sum().reset_index()
step1vs

,estado_civil_cat,sum_of_target
0,Casado,1123
1,Otros,355
2,Sin_datos,60
3,Soltero,1599


In [129]:
step3 =  step2.merge(step1vs, 
             left_on=['estado_civil_cat'], 
             right_on=['estado_civil_cat',])
step3 = pd.DataFrame(step3)
step3

,estado_civil_cat,count_of_target,sum_of_target
0,Sin_datos,17,60
1,Otros,98,355
2,Casado,295,1123
3,Soltero,434,1599


In [130]:
df_cliente['sum_of_target'].sum()

3137

In [131]:
df_cliente.dni_titular_movimiento.nunique()

844

In [132]:
print(step3['count_of_target'].sum(), step3['sum_of_target'].sum())

844 3137


In [133]:
step3['mean_est_civil'] = (step3.sum_of_target / step3.count_of_target).round(2)
step3

,estado_civil_cat,count_of_target,sum_of_target,mean_est_civil
0,Sin_datos,17,60,3.53
1,Otros,98,355,3.62
2,Casado,295,1123,3.81
3,Soltero,434,1599,3.68


In [134]:
df_cliente = df_cliente.merge(step3, 
             left_on=['estado_civil_cat'], 
             right_on=['estado_civil_cat',])
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target_x,mean_cargo,mean_sexo,count_of_target,sum_of_target_y,mean_est_civil
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,98,355,3.62
1,000e93c4f2fba984476bf86bfc483ab0732,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Alquilada,"(50.0, 60.0]",4,3.91,3.59,98,355,3.62
2,000cd731b336176daed0b381b8600a8b383,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",5,3.91,3.59,98,355,3.62
3,00067dddf31414baeece835716fddef3615,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",1,3.91,3.59,98,355,3.62
4,000b3753289033b938c52f0f1b2d55b2289,Hombre,Sector_Sin_Datos,Otros,REGION_NOROESTE,SECUNDARIOS,Propia,"(30.0, 40.0]",5,3.58,3.59,98,355,3.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,0001845f97c4f14f6c87835ed69be04c405,Mujer,Sector_Sin_Datos,Sin_datos,REGION_NOROESTE,Sin_Datos,Sin_Datos,"(60.0, 70.0]",2,3.58,3.84,17,60,3.53
840,000d1af465955c3e9d94d02597fbf814984,Mujer,Sector_Sin_Datos,Sin_datos,REGION_PATAGONIA,Sin_Datos,Sin_Datos,"(20.0, 30.0]",3,3.58,3.84,17,60,3.53
841,000cde1ad5d8114e7a5dd977930b8835659,Sin_Datos,Sector_Sin_Datos,Sin_datos,SIN_DATOS,SECUNDARIOS,Propia,Sin_Datos,3,3.58,2.57,17,60,3.53
842,000650e8144e3a42d47a21aee9e48f34804,Sin_Datos,Sector_No_Operativo,Sin_datos,SIN_DATOS,PRIMARIOS,Propia,Sin_Datos,2,4.00,2.57,17,60,3.53


In [135]:
df_cliente = df_cliente.drop(columns= ['count_of_target','sum_of_target_y']).rename({'sum_of_target_x': 'sum_of_target'}, axis=1)
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target,mean_cargo,mean_sexo,mean_est_civil
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62
1,000e93c4f2fba984476bf86bfc483ab0732,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Alquilada,"(50.0, 60.0]",4,3.91,3.59,3.62
2,000cd731b336176daed0b381b8600a8b383,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",5,3.91,3.59,3.62
3,00067dddf31414baeece835716fddef3615,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",1,3.91,3.59,3.62
4,000b3753289033b938c52f0f1b2d55b2289,Hombre,Sector_Sin_Datos,Otros,REGION_NOROESTE,SECUNDARIOS,Propia,"(30.0, 40.0]",5,3.58,3.59,3.62
...,...,...,...,...,...,...,...,...,...,...,...,...
839,0001845f97c4f14f6c87835ed69be04c405,Mujer,Sector_Sin_Datos,Sin_datos,REGION_NOROESTE,Sin_Datos,Sin_Datos,"(60.0, 70.0]",2,3.58,3.84,3.53
840,000d1af465955c3e9d94d02597fbf814984,Mujer,Sector_Sin_Datos,Sin_datos,REGION_PATAGONIA,Sin_Datos,Sin_Datos,"(20.0, 30.0]",3,3.58,3.84,3.53
841,000cde1ad5d8114e7a5dd977930b8835659,Sin_Datos,Sector_Sin_Datos,Sin_datos,SIN_DATOS,SECUNDARIOS,Propia,Sin_Datos,3,3.58,2.57,3.53
842,000650e8144e3a42d47a21aee9e48f34804,Sin_Datos,Sector_No_Operativo,Sin_datos,SIN_DATOS,PRIMARIOS,Propia,Sin_Datos,2,4.00,2.57,3.53


#### Región 

In [136]:
df_cliente.region.value_counts()

REGION_PAMPEANA     428
REGION_NOROESTE     151
REGION_CUYO         100
REGION_NORDESTE      91
REGION_PATAGONIA     64
SIN_DATOS            10
Name: region, dtype: int64

Remitimos al step1 creado para la primer variable, que contiene la suma de la variable target por cada dni_titular_movimiento.

In [137]:
step2 = df_cliente[['region', 'dni_titular_movimiento']].groupby(['region']).size().reset_index(name='count_of_target').sort_values(by='count_of_target')
step2

,region,count_of_target
5,SIN_DATOS,10
4,REGION_PATAGONIA,64
1,REGION_NORDESTE,91
0,REGION_CUYO,100
2,REGION_NOROESTE,151
3,REGION_PAMPEANA,428


In [138]:
step1vs = df_cliente[['region', 'sum_of_target']].groupby(['region']).sum().reset_index()
step1vs

,region,sum_of_target
0,REGION_CUYO,376
1,REGION_NORDESTE,347
2,REGION_NOROESTE,602
3,REGION_PAMPEANA,1539
4,REGION_PATAGONIA,248
5,SIN_DATOS,25


In [139]:
step3 =  step2.merge(step1vs, 
             left_on=['region'], 
             right_on=['region',])
step3 = pd.DataFrame(step3)
step3

,region,count_of_target,sum_of_target
0,SIN_DATOS,10,25
1,REGION_PATAGONIA,64,248
2,REGION_NORDESTE,91,347
3,REGION_CUYO,100,376
4,REGION_NOROESTE,151,602
5,REGION_PAMPEANA,428,1539


In [140]:
df_cliente['sum_of_target'].sum()

3137

In [141]:
df_cliente.dni_titular_movimiento.nunique()

844

In [142]:
print(step3['count_of_target'].sum(), step3['sum_of_target'].sum())

844 3137


In [143]:
step3['mean_reg'] = (step3.sum_of_target / step3.count_of_target).round(2)
step3

,region,count_of_target,sum_of_target,mean_reg
0,SIN_DATOS,10,25,2.50
1,REGION_PATAGONIA,64,248,3.88
2,REGION_NORDESTE,91,347,3.81
3,REGION_CUYO,100,376,3.76
4,REGION_NOROESTE,151,602,3.99
5,REGION_PAMPEANA,428,1539,3.60


In [144]:
df_cliente = df_cliente.merge(step3, 
             left_on=['region'], 
             right_on=['region',])
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target_x,mean_cargo,mean_sexo,mean_est_civil,count_of_target,sum_of_target_y,mean_reg
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62,428,1539,3.6
1,000e93c4f2fba984476bf86bfc483ab0732,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Alquilada,"(50.0, 60.0]",4,3.91,3.59,3.62,428,1539,3.6
2,000cd731b336176daed0b381b8600a8b383,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",5,3.91,3.59,3.62,428,1539,3.6
3,00067dddf31414baeece835716fddef3615,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",1,3.91,3.59,3.62,428,1539,3.6
4,000d82d4d4a5b19239d6f56cd67f3054956,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,PRIMARIOS,Propia,"(30.0, 40.0]",4,3.58,3.59,3.62,428,1539,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,00073e893b40d39e8be8bf7e74c51737985,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(30.0, 40.0]",1,3.58,3.84,3.68,10,25,2.5
840,000c4e4835abc8a679a3a7846ff2485c834,Hombre,Sector_Sin_Datos,Casado,SIN_DATOS,PRIMARIOS,Otros,"(60.0, 70.0]",1,3.58,3.59,3.81,10,25,2.5
841,000cde1ad5d8114e7a5dd977930b8835659,Sin_Datos,Sector_Sin_Datos,Sin_datos,SIN_DATOS,SECUNDARIOS,Propia,Sin_Datos,3,3.58,2.57,3.53,10,25,2.5
842,000650e8144e3a42d47a21aee9e48f34804,Sin_Datos,Sector_No_Operativo,Sin_datos,SIN_DATOS,PRIMARIOS,Propia,Sin_Datos,2,4.00,2.57,3.53,10,25,2.5


In [145]:
df_cliente = df_cliente.drop(columns= ['count_of_target','sum_of_target_y']).rename({'sum_of_target_x': 'sum_of_target'}, axis=1)
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target,mean_cargo,mean_sexo,mean_est_civil,mean_reg
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62,3.6
1,000e93c4f2fba984476bf86bfc483ab0732,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Alquilada,"(50.0, 60.0]",4,3.91,3.59,3.62,3.6
2,000cd731b336176daed0b381b8600a8b383,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",5,3.91,3.59,3.62,3.6
3,00067dddf31414baeece835716fddef3615,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",1,3.91,3.59,3.62,3.6
4,000d82d4d4a5b19239d6f56cd67f3054956,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,PRIMARIOS,Propia,"(30.0, 40.0]",4,3.58,3.59,3.62,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,00073e893b40d39e8be8bf7e74c51737985,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(30.0, 40.0]",1,3.58,3.84,3.68,2.5
840,000c4e4835abc8a679a3a7846ff2485c834,Hombre,Sector_Sin_Datos,Casado,SIN_DATOS,PRIMARIOS,Otros,"(60.0, 70.0]",1,3.58,3.59,3.81,2.5
841,000cde1ad5d8114e7a5dd977930b8835659,Sin_Datos,Sector_Sin_Datos,Sin_datos,SIN_DATOS,SECUNDARIOS,Propia,Sin_Datos,3,3.58,2.57,3.53,2.5
842,000650e8144e3a42d47a21aee9e48f34804,Sin_Datos,Sector_No_Operativo,Sin_datos,SIN_DATOS,PRIMARIOS,Propia,Sin_Datos,2,4.00,2.57,3.53,2.5


#### Nivel_estudio_descripcion_histo

In [146]:
df_cliente.nivel_estudio_descripcion_histo.value_counts()

SECUNDARIOS       441
PRIMARIOS         217
TERCIARIOS         82
Sin_Datos          53
UNIVERSITARIOS     51
Name: nivel_estudio_descripcion_histo, dtype: int64

Remitimos al step1 creado para la primer variable, que contiene la suma de la variable target por cada dni_titular_movimiento.

In [147]:
step2 = df_cliente[['nivel_estudio_descripcion_histo', 'dni_titular_movimiento']].groupby(['nivel_estudio_descripcion_histo']).size().reset_index(name='count_of_target').sort_values(by='count_of_target')
step2

,nivel_estudio_descripcion_histo,count_of_target
4,UNIVERSITARIOS,51
2,Sin_Datos,53
3,TERCIARIOS,82
0,PRIMARIOS,217
1,SECUNDARIOS,441


In [148]:
step1vs = df_cliente[['nivel_estudio_descripcion_histo', 'sum_of_target']].groupby(['nivel_estudio_descripcion_histo']).sum().reset_index()
step1vs

,nivel_estudio_descripcion_histo,sum_of_target
0,PRIMARIOS,822
1,SECUNDARIOS,1634
2,Sin_Datos,164
3,TERCIARIOS,328
4,UNIVERSITARIOS,189


In [149]:
step3 =  step2.merge(step1vs, 
             left_on=['nivel_estudio_descripcion_histo'], 
             right_on=['nivel_estudio_descripcion_histo',])
step3 = pd.DataFrame(step3)
step3

,nivel_estudio_descripcion_histo,count_of_target,sum_of_target
0,UNIVERSITARIOS,51,189
1,Sin_Datos,53,164
2,TERCIARIOS,82,328
3,PRIMARIOS,217,822
4,SECUNDARIOS,441,1634


In [150]:
df_cliente['sum_of_target'].sum()

3137

In [151]:
df_cliente.dni_titular_movimiento.nunique()

844

In [152]:
print(step3['count_of_target'].sum(), step3['sum_of_target'].sum())

844 3137


In [153]:
step3['mean_nivel_estud'] = (step3.sum_of_target / step3.count_of_target).round(2)
step3

,nivel_estudio_descripcion_histo,count_of_target,sum_of_target,mean_nivel_estud
0,UNIVERSITARIOS,51,189,3.71
1,Sin_Datos,53,164,3.09
2,TERCIARIOS,82,328,4.00
3,PRIMARIOS,217,822,3.79
4,SECUNDARIOS,441,1634,3.71


In [154]:
df_cliente = df_cliente.merge(step3, 
             left_on=['nivel_estudio_descripcion_histo'], 
             right_on=['nivel_estudio_descripcion_histo',])
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target_x,mean_cargo,mean_sexo,mean_est_civil,mean_reg,count_of_target,sum_of_target_y,mean_nivel_estud
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62,3.6,441,1634,3.71
1,000e93c4f2fba984476bf86bfc483ab0732,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Alquilada,"(50.0, 60.0]",4,3.91,3.59,3.62,3.6,441,1634,3.71
2,000cd731b336176daed0b381b8600a8b383,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",5,3.91,3.59,3.62,3.6,441,1634,3.71
3,00067dddf31414baeece835716fddef3615,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",1,3.91,3.59,3.62,3.6,441,1634,3.71
4,000b175c38b625254a81097015cdeec0551,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]",5,3.58,3.59,3.62,3.6,441,1634,3.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,0009b382e7c4e4c8ec4a0b0459ca3ff6339,Hombre,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(20.0, 30.0]",1,3.58,3.59,3.68,2.5,53,164,3.09
840,0002ae4acf8215290b73530d4820e198238,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(20.0, 30.0]",5,3.58,3.84,3.68,2.5,53,164,3.09
841,000edf1f0ef2ba6fb1c639c4fb82133b037,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(40.0, 50.0]",4,3.58,3.84,3.68,2.5,53,164,3.09
842,00080a8a903ba92d2a2a4e0cc28fa9d8765,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(30.0, 40.0]",2,3.58,3.84,3.68,2.5,53,164,3.09


In [155]:
df_cliente = df_cliente.drop(columns= ['count_of_target','sum_of_target_y']).rename({'sum_of_target_x': 'sum_of_target'}, axis=1)
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62,3.6,3.71
1,000e93c4f2fba984476bf86bfc483ab0732,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Alquilada,"(50.0, 60.0]",4,3.91,3.59,3.62,3.6,3.71
2,000cd731b336176daed0b381b8600a8b383,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",5,3.91,3.59,3.62,3.6,3.71
3,00067dddf31414baeece835716fddef3615,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(50.0, 60.0]",1,3.91,3.59,3.62,3.6,3.71
4,000b175c38b625254a81097015cdeec0551,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(40.0, 50.0]",5,3.58,3.59,3.62,3.6,3.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,0009b382e7c4e4c8ec4a0b0459ca3ff6339,Hombre,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(20.0, 30.0]",1,3.58,3.59,3.68,2.5,3.09
840,0002ae4acf8215290b73530d4820e198238,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(20.0, 30.0]",5,3.58,3.84,3.68,2.5,3.09
841,000edf1f0ef2ba6fb1c639c4fb82133b037,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(40.0, 50.0]",4,3.58,3.84,3.68,2.5,3.09
842,00080a8a903ba92d2a2a4e0cc28fa9d8765,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(30.0, 40.0]",2,3.58,3.84,3.68,2.5,3.09


#### Client_edad_cat

In [156]:
df_cliente.client_edad_cat.value_counts()

(30.0, 40.0]     205
(40.0, 50.0]     195
(50.0, 60.0]     131
(60.0, 70.0]     125
(70.0, 110.0]     99
(20.0, 30.0]      85
Sin_Datos          4
Name: client_edad_cat, dtype: int64

Remitimos al step1 creado para la primer variable, que contiene la suma de la variable target por cada dni_titular_movimiento.

In [157]:
step2 = df_cliente[['client_edad_cat', 'dni_titular_movimiento']].groupby(['client_edad_cat']).size().reset_index(name='count_of_target').sort_values(by='count_of_target')
step2

,client_edad_cat,count_of_target
6,Sin_Datos,4
0,"(20.0, 30.0]",85
5,"(70.0, 110.0]",99
4,"(60.0, 70.0]",125
3,"(50.0, 60.0]",131
2,"(40.0, 50.0]",195
1,"(30.0, 40.0]",205


In [158]:
step1vs = df_cliente[['client_edad_cat', 'sum_of_target']].groupby(['client_edad_cat']).sum().reset_index()
step1vs

,client_edad_cat,sum_of_target
0,"(20.0, 30.0]",296
1,"(30.0, 40.0]",799
2,"(40.0, 50.0]",740
3,"(50.0, 60.0]",491
4,"(60.0, 70.0]",455
5,"(70.0, 110.0]",347
6,Sin_Datos,9


In [159]:
step3 =  step2.merge(step1vs, 
             left_on=['client_edad_cat'], 
             right_on=['client_edad_cat',])
step3 = pd.DataFrame(step3)
step3

,client_edad_cat,count_of_target,sum_of_target
0,Sin_Datos,4,9
1,"(20.0, 30.0]",85,296
2,"(70.0, 110.0]",99,347
3,"(60.0, 70.0]",125,455
4,"(50.0, 60.0]",131,491
5,"(40.0, 50.0]",195,740
6,"(30.0, 40.0]",205,799


In [160]:
df_cliente['sum_of_target'].sum()

3137

In [161]:
df_cliente.dni_titular_movimiento.nunique()

844

In [162]:
print(step3['count_of_target'].sum(), step3['sum_of_target'].sum())

844 3137


In [163]:
step3['mean_edad'] = (step3.sum_of_target / step3.count_of_target).round(2)
step3

,client_edad_cat,count_of_target,sum_of_target,mean_edad
0,Sin_Datos,4,9,2.25
1,"(20.0, 30.0]",85,296,3.48
2,"(70.0, 110.0]",99,347,3.51
3,"(60.0, 70.0]",125,455,3.64
4,"(50.0, 60.0]",131,491,3.75
5,"(40.0, 50.0]",195,740,3.79
6,"(30.0, 40.0]",205,799,3.90


In [164]:
df_cliente = df_cliente.merge(step3, 
             left_on=['client_edad_cat'], 
             right_on=['client_edad_cat',])
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target_x,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,count_of_target,sum_of_target_y,mean_edad
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62,3.6,3.71,125,455,3.64
1,00014f363751f71db985d4b286eb8fe7248,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",1,3.58,3.59,3.62,3.6,3.71,125,455,3.64
2,000b5dab881f5d597c9633417545dfed492,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.58,3.59,3.62,3.6,3.71,125,455,3.64
3,0001f2e0ecebd8b0f2dd7592a16a9c67829,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Sin_Datos,"(60.0, 70.0]",1,3.58,3.59,3.62,3.6,3.71,125,455,3.64
4,000429bc0c260af377a8f7d0701a392a646,Mujer,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.91,3.84,3.62,3.6,3.71,125,455,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,00073e893b40d39e8be8bf7e74c51737985,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(30.0, 40.0]",1,3.58,3.84,3.68,2.5,3.09,205,799,3.90
840,000cde1ad5d8114e7a5dd977930b8835659,Sin_Datos,Sector_Sin_Datos,Sin_datos,SIN_DATOS,SECUNDARIOS,Propia,Sin_Datos,3,3.58,2.57,3.53,2.5,3.71,4,9,2.25
841,000d5e14adcc2aab78342c3aa9b5e4e9782,Hombre,Sector_Sin_Datos,Soltero,REGION_PAMPEANA,PRIMARIOS,Sin_Datos,Sin_Datos,3,3.58,3.59,3.68,3.6,3.79,4,9,2.25
842,000650e8144e3a42d47a21aee9e48f34804,Sin_Datos,Sector_No_Operativo,Sin_datos,SIN_DATOS,PRIMARIOS,Propia,Sin_Datos,2,4.00,2.57,3.53,2.5,3.79,4,9,2.25


In [165]:
df_cliente = df_cliente.drop(columns= ['count_of_target','sum_of_target_y']).rename({'sum_of_target_x': 'sum_of_target'}, axis=1)
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,mean_edad
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62,3.6,3.71,3.64
1,00014f363751f71db985d4b286eb8fe7248,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",1,3.58,3.59,3.62,3.6,3.71,3.64
2,000b5dab881f5d597c9633417545dfed492,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.58,3.59,3.62,3.6,3.71,3.64
3,0001f2e0ecebd8b0f2dd7592a16a9c67829,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Sin_Datos,"(60.0, 70.0]",1,3.58,3.59,3.62,3.6,3.71,3.64
4,000429bc0c260af377a8f7d0701a392a646,Mujer,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.91,3.84,3.62,3.6,3.71,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,00073e893b40d39e8be8bf7e74c51737985,Mujer,Sector_Sin_Datos,Soltero,SIN_DATOS,Sin_Datos,Sin_Datos,"(30.0, 40.0]",1,3.58,3.84,3.68,2.5,3.09,3.90
840,000cde1ad5d8114e7a5dd977930b8835659,Sin_Datos,Sector_Sin_Datos,Sin_datos,SIN_DATOS,SECUNDARIOS,Propia,Sin_Datos,3,3.58,2.57,3.53,2.5,3.71,2.25
841,000d5e14adcc2aab78342c3aa9b5e4e9782,Hombre,Sector_Sin_Datos,Soltero,REGION_PAMPEANA,PRIMARIOS,Sin_Datos,Sin_Datos,3,3.58,3.59,3.68,3.6,3.79,2.25
842,000650e8144e3a42d47a21aee9e48f34804,Sin_Datos,Sector_No_Operativo,Sin_datos,SIN_DATOS,PRIMARIOS,Propia,Sin_Datos,2,4.00,2.57,3.53,2.5,3.79,2.25


#### Rel_vivienda_descripcion_histo

In [166]:
df_cliente.rel_vivienda_descripcion_histo.value_counts()

Propia         557
Sin_Datos      131
De familiar     67
Otros           66
Alquilada       23
Name: rel_vivienda_descripcion_histo, dtype: int64

Remitimos al step1 creado para la primer variable, que contiene la suma de la variable target por cada dni_titular_movimiento.

In [167]:
step2 = df_cliente[['rel_vivienda_descripcion_histo', 'dni_titular_movimiento']].groupby(['rel_vivienda_descripcion_histo']).size().reset_index(name='count_of_target').sort_values(by='count_of_target')
step2

,rel_vivienda_descripcion_histo,count_of_target
0,Alquilada,23
2,Otros,66
1,De familiar,67
4,Sin_Datos,131
3,Propia,557


In [168]:
step1vs = df_cliente[['rel_vivienda_descripcion_histo', 'sum_of_target']].groupby(['rel_vivienda_descripcion_histo']).sum().reset_index()
step1vs

,rel_vivienda_descripcion_histo,sum_of_target
0,Alquilada,87
1,De familiar,250
2,Otros,264
3,Propia,2091
4,Sin_Datos,445


In [169]:
step3 =  step2.merge(step1vs, 
             left_on=['rel_vivienda_descripcion_histo'], 
             right_on=['rel_vivienda_descripcion_histo',])
step3 = pd.DataFrame(step3)
step3

,rel_vivienda_descripcion_histo,count_of_target,sum_of_target
0,Alquilada,23,87
1,Otros,66,264
2,De familiar,67,250
3,Sin_Datos,131,445
4,Propia,557,2091


In [170]:
df_cliente['sum_of_target'].sum()

3137

In [171]:
df_cliente.dni_titular_movimiento.nunique()

844

In [172]:
print(step3['count_of_target'].sum(), step3['sum_of_target'].sum())

844 3137


In [173]:
step3['mean_est_viv'] = (step3.sum_of_target / step3.count_of_target).round(2)
step3

,rel_vivienda_descripcion_histo,count_of_target,sum_of_target,mean_est_viv
0,Alquilada,23,87,3.78
1,Otros,66,264,4.00
2,De familiar,67,250,3.73
3,Sin_Datos,131,445,3.40
4,Propia,557,2091,3.75


In [174]:
df_cliente = df_cliente.merge(step3, 
             left_on=['rel_vivienda_descripcion_histo'], 
             right_on=['rel_vivienda_descripcion_histo',])
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target_x,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,mean_edad,count_of_target,sum_of_target_y,mean_est_viv
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62,3.60,3.71,3.64,557,2091,3.75
1,00014f363751f71db985d4b286eb8fe7248,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",1,3.58,3.59,3.62,3.60,3.71,3.64,557,2091,3.75
2,000b5dab881f5d597c9633417545dfed492,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.58,3.59,3.62,3.60,3.71,3.64,557,2091,3.75
3,000429bc0c260af377a8f7d0701a392a646,Mujer,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.91,3.84,3.62,3.60,3.71,3.64,557,2091,3.75
4,0009d6e7ad9d5c429c0de8d0b2ac8c8b439,Mujer,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.91,3.84,3.62,3.60,3.71,3.64,557,2091,3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,000abce27a319dd5c0eaefbbb844006e588,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,SECUNDARIOS,Alquilada,"(30.0, 40.0]",1,3.91,3.84,3.68,3.60,3.71,3.90,23,87,3.78
840,00052e99dd88efc172a46470a94b1e58036,Hombre,Sector_Seguridad,Soltero,REGION_NOROESTE,SECUNDARIOS,Alquilada,"(30.0, 40.0]",2,3.65,3.59,3.68,3.99,3.71,3.90,23,87,3.78
841,0009d010e4faf69552a814a33832b185877,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Alquilada,"(30.0, 40.0]",4,3.91,3.84,3.68,3.60,3.71,3.90,23,87,3.78
842,00045ea05f2154e51d959ab394f03aca861,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Alquilada,"(30.0, 40.0]",5,3.91,3.84,3.68,3.60,3.71,3.90,23,87,3.78


In [175]:
df_cliente = df_cliente.drop(columns= ['count_of_target','sum_of_target_y']).rename({'sum_of_target_x': 'sum_of_target'}, axis=1)
df_cliente

,dni_titular_movimiento,sexo_descripcion,cargo_sector_desc_hist,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,client_edad_cat,sum_of_target,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,mean_edad,mean_est_viv
0,0001686b52949b5461ffcbc766687e45031,Hombre,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",7,3.91,3.59,3.62,3.60,3.71,3.64,3.75
1,00014f363751f71db985d4b286eb8fe7248,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",1,3.58,3.59,3.62,3.60,3.71,3.64,3.75
2,000b5dab881f5d597c9633417545dfed492,Hombre,Sector_Sin_Datos,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.58,3.59,3.62,3.60,3.71,3.64,3.75
3,000429bc0c260af377a8f7d0701a392a646,Mujer,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.91,3.84,3.62,3.60,3.71,3.64,3.75
4,0009d6e7ad9d5c429c0de8d0b2ac8c8b439,Mujer,Sector_Empleado_Comercio,Otros,REGION_PAMPEANA,SECUNDARIOS,Propia,"(60.0, 70.0]",3,3.91,3.84,3.62,3.60,3.71,3.64,3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,000abce27a319dd5c0eaefbbb844006e588,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,SECUNDARIOS,Alquilada,"(30.0, 40.0]",1,3.91,3.84,3.68,3.60,3.71,3.90,3.78
840,00052e99dd88efc172a46470a94b1e58036,Hombre,Sector_Seguridad,Soltero,REGION_NOROESTE,SECUNDARIOS,Alquilada,"(30.0, 40.0]",2,3.65,3.59,3.68,3.99,3.71,3.90,3.78
841,0009d010e4faf69552a814a33832b185877,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Alquilada,"(30.0, 40.0]",4,3.91,3.84,3.68,3.60,3.71,3.90,3.78
842,00045ea05f2154e51d959ab394f03aca861,Mujer,Sector_Empleado_Comercio,Soltero,REGION_PAMPEANA,UNIVERSITARIOS,Alquilada,"(30.0, 40.0]",5,3.91,3.84,3.68,3.60,3.71,3.90,3.78


In [176]:
df_enc_cliente = df_cliente[['dni_titular_movimiento','mean_cargo','mean_sexo',
            'mean_est_civil','mean_reg', 'mean_nivel_estud','mean_est_viv', 'mean_edad']]
df_enc_cliente

,dni_titular_movimiento,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,mean_est_viv,mean_edad
0,0001686b52949b5461ffcbc766687e45031,3.91,3.59,3.62,3.60,3.71,3.75,3.64
1,00014f363751f71db985d4b286eb8fe7248,3.58,3.59,3.62,3.60,3.71,3.75,3.64
2,000b5dab881f5d597c9633417545dfed492,3.58,3.59,3.62,3.60,3.71,3.75,3.64
3,000429bc0c260af377a8f7d0701a392a646,3.91,3.84,3.62,3.60,3.71,3.75,3.64
4,0009d6e7ad9d5c429c0de8d0b2ac8c8b439,3.91,3.84,3.62,3.60,3.71,3.75,3.64
...,...,...,...,...,...,...,...,...
839,000abce27a319dd5c0eaefbbb844006e588,3.91,3.84,3.68,3.60,3.71,3.78,3.90
840,00052e99dd88efc172a46470a94b1e58036,3.65,3.59,3.68,3.99,3.71,3.78,3.90
841,0009d010e4faf69552a814a33832b185877,3.91,3.84,3.68,3.60,3.71,3.78,3.90
842,00045ea05f2154e51d959ab394f03aca861,3.91,3.84,3.68,3.60,3.71,3.78,3.90


In [177]:
df_obj_mensual = df_obj.merge(df_enc_cliente, 
             left_on=['dni_titular_movimiento'], 
             right_on=['dni_titular_movimiento',])
df_obj_mensual

,dni_titular_movimiento,anio_mes_cupon,client_edad,client_antig,client_domicilio_codigo_postal,trans__x0_AV,trans__x0_AX,trans__x0_EX,trans__x0_MC,trans__x0_PC,...,monto_ajustado,fg_aumentado,o_mes,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,mean_est_viv,mean_edad
0,000000b5aea2c9ea7cc155f6ebcef97f826,202007,47.0,224.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0,0,3.58,3.84,3.68,3.60,4.00,3.75,3.79
1,000000b5aea2c9ea7cc155f6ebcef97f826,202008,47.0,225.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,-4.72,0,1,3.58,3.84,3.68,3.60,4.00,3.75,3.79
2,000000b5aea2c9ea7cc155f6ebcef97f826,202009,47.0,226.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,-4.61,0,2,3.58,3.84,3.68,3.60,4.00,3.75,3.79
3,000000b5aea2c9ea7cc155f6ebcef97f826,202010,47.0,227.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,-4.46,0,3,3.58,3.84,3.68,3.60,4.00,3.75,3.79
4,000000b5aea2c9ea7cc155f6ebcef97f826,202011,47.0,228.0,3000.0,0.0,0.0,0.0,0.0,0.0,...,-4.34,0,4,3.58,3.84,3.68,3.60,4.00,3.75,3.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10123,000f0b73ebfa002a79a0642b82e87919904,202102,64.0,22.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,12622.42,0,7,3.58,3.84,3.81,3.76,3.71,3.40,3.64
10124,000f0b73ebfa002a79a0642b82e87919904,202103,64.0,23.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,2716.90,0,8,3.58,3.84,3.81,3.76,3.71,3.40,3.64
10125,000f0b73ebfa002a79a0642b82e87919904,202104,64.0,24.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,5713.04,1,9,3.58,3.84,3.81,3.76,3.71,3.40,3.64
10126,000f0b73ebfa002a79a0642b82e87919904,202105,64.0,25.0,5730.0,0.0,0.0,0.0,0.0,0.0,...,2063.81,0,10,3.58,3.84,3.81,3.76,3.71,3.40,3.64


In [178]:
df_obj_diario = df_encode.merge(df_enc_cliente, 
             left_on=['dni_titular_movimiento'], 
             right_on=['dni_titular_movimiento',])
df_obj_diario

,dni_titular_movimiento,anio_mes_cupon,client_edad,client_antig,client_domicilio_codigo_postal,fecha_cupon_movimiento,trans__x0_AV,trans__x0_AX,trans__x0_EX,trans__x0_MC,...,trans__x3_8,trans__x3_9,monto_ajustado,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,mean_est_viv,mean_edad
0,00017c577769060500211670502411b5913,202007,63.0,79,3460,15937344.0,0.0,0.0,0.0,0.0,...,0.0,1.0,-3445.56,3.77,3.84,3.81,3.81,3.79,3.75,3.64
1,00017c577769060500211670502411b5913,202008,63.0,80,3460,15964992.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3094.61,3.77,3.84,3.81,3.81,3.79,3.75,3.64
2,00017c577769060500211670502411b5913,202008,63.0,80,3460,15971904.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1417.27,3.77,3.84,3.81,3.81,3.79,3.75,3.64
3,00017c577769060500211670502411b5913,202008,63.0,80,3460,15971904.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1282.05,3.77,3.84,3.81,3.81,3.79,3.75,3.64
4,00017c577769060500211670502411b5913,202008,63.0,80,3460,15971904.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4209.55,3.77,3.84,3.81,3.81,3.79,3.75,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65378,00073e893b40d39e8be8bf7e74c51737985,202104,36.0,0,4000,16197408.0,0.0,0.0,0.0,0.0,...,0.0,0.0,558.19,3.58,3.84,3.68,2.50,3.09,3.40,3.90
65379,0009a4bc8fa0602c8c05d09e26008a19599,202104,24.0,3,4200,16189632.0,0.0,0.0,0.0,0.0,...,0.0,0.0,130.29,3.58,3.59,3.68,3.99,3.09,3.40,3.48
65380,0009a4bc8fa0602c8c05d09e26008a19599,202104,24.0,3,4200,16189632.0,0.0,0.0,0.0,0.0,...,0.0,0.0,130.29,3.58,3.59,3.68,3.99,3.09,3.40,3.48
65381,0009a4bc8fa0602c8c05d09e26008a19599,202104,24.0,3,4200,16193088.0,0.0,0.0,0.0,0.0,...,0.0,0.0,247.06,3.58,3.59,3.68,3.99,3.09,3.40,3.48


### 3.4 Creación de variable distancia según domicilio_código_postal

Procedemos a crear una variable que mida la distancia entre los distintos códigos postales. En nuestro caso, al ser Tarjeta Naranja de la ciudad de Córdoba, usaremos de referencia el codigo postal de dicha ciudad. 

In [179]:
df_obj_mensual.client_domicilio_codigo_postal.describe()

count    10128.000000
mean      4390.601896
std       1988.278985
min       1000.000000
25%       3100.000000
50%       4304.000000
75%       5507.500000
max       9420.000000
Name: client_domicilio_codigo_postal, dtype: float64

In [180]:
df[df["ciudad"] == "CORDOBA"]['domicilio_codigo_postal'].value_counts()

5000    1114
5006     881
5016     760
5014     646
5009     614
5003     480
5008     359
5002     300
5012     291
5017     264
5021     177
5004     174
5020     149
5011     132
5022     132
5001     126
5013      61
Name: domicilio_codigo_postal, dtype: int64

In [181]:
dist = pgeocode.GeoDistance('ar')
# distancia aprox. entre jujuy y cordoba en Km
dist.query_postal_code("4600", "5000")

809.05828514463

In [182]:
def distancia(cod_post):
    return pgeocode.GeoDistance('ar').query_postal_code(cod_post, "5000")

#df['domicilio_codigo_postal'].apply(distancia)

In [183]:
df_obj_mensual['client_domicilio_codigo_postal']

0        3000.0
1        3000.0
2        3000.0
3        3000.0
4        3000.0
          ...  
10123    5730.0
10124    5730.0
10125    5730.0
10126    5730.0
10127    5730.0
Name: client_domicilio_codigo_postal, Length: 10128, dtype: float64

In [184]:
pd.Series(['2200', '4181', '4200'])

0    2200
1    4181
2    4200
dtype: object

In [185]:
df_obj_mensual['client_domicilio_codigo_postal'].astype(int).astype(str)

0        3000
1        3000
2        3000
3        3000
4        3000
         ... 
10123    5730
10124    5730
10125    5730
10126    5730
10127    5730
Name: client_domicilio_codigo_postal, Length: 10128, dtype: object

In [186]:
df_cp = df_obj_mensual['client_domicilio_codigo_postal'].astype(int).astype(str).drop_duplicates().reset_index().drop(columns='index')
df_cp['client_domicilio_codigo_postal']

0      3000
1      5541
2      1754
3      4700
4      5000
       ... 
372    4603
373    1834
374    1885
375    5594
376    4650
Name: client_domicilio_codigo_postal, Length: 377, dtype: object

In [187]:
def distancia(cod_post):
    return pgeocode.GeoDistance('ar').query_postal_code(cod_post, "5000")

df_cp['geodist_cp'] = df_cp['client_domicilio_codigo_postal'].apply(distancia).round(2)

In [188]:
df_cp.loc[df_cp[df_cp['geodist_cp'].isna()].index.values,'geodist_cp'] = pd.Series(df_cp[df_cp['geodist_cp'].isna()]['client_domicilio_codigo_postal'].str[:2]+'00').apply(distancia).round(2)

In [189]:
df_cp.loc[df_cp[df_cp['geodist_cp'].isna()].index.values,'geodist_cp'] = pd.Series(df_cp[df_cp['geodist_cp'].isna()]['client_domicilio_codigo_postal'].str[:1]+'000').apply(distancia).round(2)

In [190]:
df[df['provincia'] == 'BUENOS AIRES']['domicilio_codigo_postal']

0         1766
2         7107
18        1744
28        1625
38        1891
          ... 
124253    1650
124269    1826
124285    1824
124286    1824
124289    1757
Name: domicilio_codigo_postal, Length: 10337, dtype: object

In [191]:
df_cp['geodist_cp'].fillna(644.06, inplace=True) # Todos los que quedan son CABA

In [192]:
df_cp.isnull().sum()

client_domicilio_codigo_postal    0
geodist_cp                        0
dtype: int64

In [193]:
df_obj_mensual['client_domicilio_codigo_postal'] = df_obj_mensual['client_domicilio_codigo_postal'].astype(int).astype(str)
df_obj_diario['client_domicilio_codigo_postal'] = df_obj_diario['client_domicilio_codigo_postal'].astype(int).astype(str)

In [194]:
df_obj_mensual = df_obj_mensual.merge(df_cp, on='client_domicilio_codigo_postal')

In [195]:
df_obj_mensual

,dni_titular_movimiento,anio_mes_cupon,client_edad,client_antig,client_domicilio_codigo_postal,trans__x0_AV,trans__x0_AX,trans__x0_EX,trans__x0_MC,trans__x0_PC,...,fg_aumentado,o_mes,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,mean_est_viv,mean_edad,geodist_cp
0,000000b5aea2c9ea7cc155f6ebcef97f826,202007,47.0,224.0,3000,0.0,0.0,0.0,0.0,0.0,...,0,0,3.58,3.84,3.68,3.60,4.0,3.75,3.79,329.37
1,000000b5aea2c9ea7cc155f6ebcef97f826,202008,47.0,225.0,3000,0.0,0.0,0.0,0.0,0.0,...,0,1,3.58,3.84,3.68,3.60,4.0,3.75,3.79,329.37
2,000000b5aea2c9ea7cc155f6ebcef97f826,202009,47.0,226.0,3000,0.0,0.0,0.0,0.0,0.0,...,0,2,3.58,3.84,3.68,3.60,4.0,3.75,3.79,329.37
3,000000b5aea2c9ea7cc155f6ebcef97f826,202010,47.0,227.0,3000,0.0,0.0,0.0,0.0,0.0,...,0,3,3.58,3.84,3.68,3.60,4.0,3.75,3.79,329.37
4,000000b5aea2c9ea7cc155f6ebcef97f826,202011,47.0,228.0,3000,0.0,0.0,0.0,0.0,0.0,...,0,4,3.58,3.84,3.68,3.60,4.0,3.75,3.79,329.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10123,000ef071e95351afc1149fa1afe4862a079,202102,31.0,64.0,4650,0.0,0.0,0.0,0.0,0.0,...,1,7,3.58,3.59,3.68,3.99,4.0,3.75,3.90,1048.33
10124,000ef071e95351afc1149fa1afe4862a079,202103,31.0,65.0,4650,0.0,0.0,0.0,0.0,0.0,...,0,8,3.58,3.59,3.68,3.99,4.0,3.75,3.90,1048.33
10125,000ef071e95351afc1149fa1afe4862a079,202104,31.0,66.0,4650,0.0,0.0,0.0,0.0,0.0,...,0,9,3.58,3.59,3.68,3.99,4.0,3.75,3.90,1048.33
10126,000ef071e95351afc1149fa1afe4862a079,202105,31.0,67.0,4650,0.0,0.0,0.0,0.0,0.0,...,1,10,3.58,3.59,3.68,3.99,4.0,3.75,3.90,1048.33


In [196]:
df_obj_diario = df_obj_diario.merge(df_cp, on='client_domicilio_codigo_postal')

In [197]:
df_obj_diario

,dni_titular_movimiento,anio_mes_cupon,client_edad,client_antig,client_domicilio_codigo_postal,fecha_cupon_movimiento,trans__x0_AV,trans__x0_AX,trans__x0_EX,trans__x0_MC,...,trans__x3_9,monto_ajustado,mean_cargo,mean_sexo,mean_est_civil,mean_reg,mean_nivel_estud,mean_est_viv,mean_edad,geodist_cp
0,00017c577769060500211670502411b5913,202007,63.0,79,3460,15937344.0,0.0,0.0,0.0,0.0,...,1.0,-3445.56,3.77,3.84,3.81,3.81,3.79,3.75,3.64,621.15
1,00017c577769060500211670502411b5913,202008,63.0,80,3460,15964992.0,0.0,0.0,0.0,0.0,...,1.0,3094.61,3.77,3.84,3.81,3.81,3.79,3.75,3.64,621.15
2,00017c577769060500211670502411b5913,202008,63.0,80,3460,15971904.0,0.0,0.0,0.0,0.0,...,1.0,1417.27,3.77,3.84,3.81,3.81,3.79,3.75,3.64,621.15
3,00017c577769060500211670502411b5913,202008,63.0,80,3460,15971904.0,0.0,0.0,0.0,0.0,...,1.0,1282.05,3.77,3.84,3.81,3.81,3.79,3.75,3.64,621.15
4,00017c577769060500211670502411b5913,202008,63.0,80,3460,15971904.0,0.0,0.0,0.0,0.0,...,0.0,4209.55,3.77,3.84,3.81,3.81,3.79,3.75,3.64,621.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65378,000d5d32bde0fc5d534edc79aae996c2302,202103,29.0,0,5632,16170624.0,0.0,0.0,0.0,0.0,...,0.0,2639.17,3.58,3.84,3.68,3.76,3.09,3.40,3.48,511.81
65379,000d5d32bde0fc5d534edc79aae996c2302,202104,29.0,1,5632,16185312.0,0.0,0.0,0.0,0.0,...,0.0,411.15,3.58,3.84,3.68,3.76,3.09,3.40,3.48,511.81
65380,000d5d32bde0fc5d534edc79aae996c2302,202104,29.0,1,5632,16185312.0,0.0,0.0,0.0,0.0,...,0.0,411.15,3.58,3.84,3.68,3.76,3.09,3.40,3.48,511.81
65381,000d5d32bde0fc5d534edc79aae996c2302,202104,29.0,1,5632,16193088.0,0.0,0.0,0.0,0.0,...,0.0,247.06,3.58,3.84,3.68,3.76,3.09,3.40,3.48,511.81


In [198]:
df_obj_mensual.to_parquet('preprocesado_no_supervisado_agrupado.parquet')

In [199]:
df_obj_diario.to_parquet('preprocesado_no_supervisado_diario.parquet')